# DID players dataframes

En este script nos dedicaremos a crear una base de datos limpia segmentada por hitters y fielders. Se divide en las siguientes secciones:

- **Visualización del contenido de las bases de datos.**
- **Limpieza de la base de datos y exportación.**
- **Construcción de variables para estimaciones.**
- **Unión de las bases de datos para nuevas bases transversales.**
- **Segmentación de bases datos de acuerdo a la agencia libre.**
- **Realización del panel data de acuerdo a las especificaciones.**
- **Generación de las variables para el modelo dinámico.**

Importemos los modulos necesarios así como especificar la configuración deseada.

In [1]:
import pandas as pd
import numpy as np
import math
import os
import warnings
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from matplotlib.colors import ListedColormap
from termcolor import colored
print('Modulos importados')

Modulos importados


In [2]:
# Configuraciones
warnings.filterwarnings('ignore')

In [3]:
# Directorio de trabajo
print("Directorio de trabajo previo: " + str(os.getcwd()))
# Cambiemoslo
os.chdir('/home/usuario/Documentos/Github/Proyectos/MLB_HN/')

Directorio de trabajo previo: /home/usuario/Documentos/Github/Proyectos/MLB_HN/ETL_Scripts/Players/DID


In [4]:
# Veamos el directorio actual de trabajo
print(os.getcwd())
# El directorio anterior es el correcto, pero si no lo fuese, hacemos lo sigueinte:
path = '/home/usuario/Documentos/Github/Proyectos/MLB_HN'
print("Nuevo directorio de trabajo: " + str(os.chdir(path)))

/home/usuario/Documentos/Github/Proyectos/MLB_HN
Nuevo directorio de trabajo: None


## Visualización de las bases de datos

### Equipos por estado

In [5]:
states = 'Data/Teams/team_states.csv'
df_states = pd.read_csv(states)

In [6]:
df_states.head()

,Estado,Cantidad de equipos
0,Alabama,0
1,Alaska,0
2,Arizona,1
3,Arkansas,0
4,California,5


### Acrónimos

Nos servirá como llave intermedia para unificar las bases de datos de los equipos

In [7]:
acronym = 'Data/Teams/team_acronym.csv'
df_acronym = pd.read_csv(acronym)

In [8]:
df_acronym.head()

,Equipo,Acronimo,Estado
0,Arizona Diamondbacks,ARI,Arizona
1,Atlanta Braves,ATL,Georgia
2,Baltimore Orioles,BAL,Maryland
3,Boston Red Sox,BOS,Massachusetts
4,Chicago Cubs,CHC,Illinois


Unamos esta dataframe con el de los equipos por estado

In [9]:
acronym_state = pd.merge(df_states, df_acronym, on = 'Estado')

In [10]:
acronym_state.head()

,Estado,Cantidad de equipos,Equipo,Acronimo
0,Arizona,1,Arizona Diamondbacks,ARI
1,California,5,Los Angeles Angels,LAA
2,California,5,Los Angeles Dodgers,LAD
3,California,5,Oakland Athletics,OAK
4,California,5,San Diego Padres,SD


En este caso, el nombre de las variables es claro

## Algoritmo para la creación de las bases de datos

A continuaicón, se optimizará el código para que se puedan obtener los *dataframes* anteriores para un cojuntos de datos de años secuenciales, como es nuestro caso

In [11]:
# Auxiliares:
free_agents = 'Data/Free_Agents/free_agents_'
hitting = 'Data/Statistics/Cumulative/Hitting/hitting_'
pitching = 'Data/Statistics/Cumulative/Pitching/pitching_'
salary = 'Data/Salary/salary_'
teams = 'ETL_Data/Transversal/Teams/free_agents_team_'
csv = '.csv'
period = 12
# Originales:
df_free_agents = [None]*period
df_hitting = [None]*period
df_pitching = [None]*period
df_salary = [None]*period
df_teams = [None]*period
# Copias:
df_free_agents_copy = [None]*period
df_hitting_copy = [None]*period
df_pitching_copy = [None]*period
df_salary_copy = [None]*period
df_teams_copy = [None]*period
# Producto final:
df_pitchers = [None]*period
df_hitters = [None]*period
df_pitchers_free_agents = [None]*period
df_hitters_free_agents = [None]*period
df_pitchers_no_free_agents = [None]*period
df_hitters_no_free_agents = [None]*period
df_panel_hitters = [None]*period
df_panel_pitchers = [None]*period

Leamos todos los archivos y creemos las copias

In [12]:
for year in range(0,period):    
    df_free_agents[year] = pd.read_csv(free_agents + str(2011 + year) + csv)
    df_hitting[year] = pd.read_csv(hitting + str(2011 + year) + csv)
    df_pitching[year] = pd.read_csv(pitching + str(2011 + year) + csv)
    df_salary[year] = pd.read_csv(salary + str(2011 + year) + csv)
    df_teams[year] = pd.read_csv(teams + str(2011 + year) + csv)
    
    df_free_agents_copy[year] = df_free_agents[year].copy()
    df_hitting_copy[year] = df_hitting[year].copy()
    df_pitching_copy[year] = df_pitching[year].copy()
    df_salary_copy[year] = df_salary[year].copy()
    df_teams_copy[year] = pd.read_csv(teams + str(2011 + year) + csv)

Tratemos las bases de datos por separado. Sin embargo, a todas les quitaremos la columna de rango y *Cash2023*.

Como no queremos que se repita la columna del año de la temporada de la base de datos, borremos la columna de *Year* de la base  de datos de los agentes libres. Como los años del contrato aparecen en la base de datos sobre los salarios, se prefiere conservar dicha columna en la base de datos de salarios puesto que esta base de datos es más general que la de los agentes libres, razón por la que se borrará de esta última base de datos. 

El equipo al que se cambia el agente libre está señalado por la columna del equipo en la base de datos de salarios y la estadísticas deportivas por lo que se borrará *Team From To* de la base de datos de los agentes libres. 

Como nos importan los salarios para este analisis, quitaremos la columna de los equipos en las bases de datos sobre las estadísticas deportivas de todos los jugadores, así como la posición que ocupan.

In [13]:
for year in range(0,period):
    # Drop columns:
    if any(name in df_free_agents_copy[year].columns for name in ['Rank','Pos','Year','Team From To']):
        df_free_agents_copy[year].drop('Rank', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Year', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Pos', axis = 1, inplace = True)
        df_free_agents_copy[year].drop('Team From To', axis = 1, inplace = True)
    if 'Rank' in df_salary_copy[year].columns:
        df_salary_copy[year].drop('Rank', axis = 1, inplace = True)
    if any(name in df_hitting_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_hitting_copy[year].drop('Rank', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Team', axis = 1, inplace = True)
        df_hitting_copy[year].drop('Pos', axis = 1, inplace = True)
    if any(name in df_pitching_copy[year].columns for name in ['Rank','Year','Cash2023','Team','Pos']):
        df_pitching_copy[year].drop('Rank', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Cash2023', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Team', axis = 1, inplace = True)
        df_pitching_copy[year].drop('Pos', axis = 1, inplace = True)

Debido a que aparecen columnas que inician con el  nombre *Unnamed*, tendremos que borrarlas con algún método general, el cual se muestra a continuación:

In [14]:
for year in range(0,period):
    # Base de datos de agentes libres:
    df_free_agents_copy[year].drop(df_free_agents_copy[year].columns[df_free_agents_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los salarios:
    df_salary_copy[year].drop(df_salary_copy[year].columns[df_salary_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los hitters:
    df_hitting_copy[year].drop(df_hitting_copy[year].columns[df_hitting_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
    
    # Base de datos de los pitchers:
    df_pitching_copy[year].drop(df_pitching_copy[year].columns[df_pitching_copy[year].columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

Verifiquemos que ya no se encuentran dichas columnas molestas

In [15]:
df_free_agents_copy[9].columns

Index(['Player', 'Status', 'Team From', 'YRS', 'Value', 'AAV'], dtype='object')

In [16]:
df_salary_copy[11].columns

Index(['Player', 'Year', 'Pos', 'Team', 'BaseSalary', 'SigningBonus',
       'Payroll Salary', 'Adj Salary', 'Salary%', 'Cash', 'AAV', 'CONT YR',
       'CONT VALUE', 'Earnings', 'FA Year', 'Sign Age', 'Age', 'Weight',
       'Height'],
      dtype='object')

In [17]:
df_hitting_copy[2].columns

Index(['Player', 'GP', 'GP%', 'GS', 'GS%', 'AB', 'H', '2B', '3B', 'HR', 'RBI',
       'AVG', 'OBP', 'SLG', 'OPS', 'WAR', 'TVS'],
      dtype='object')

In [18]:
df_pitching_copy[5].columns

Index(['Player', 'GP', 'GS', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'W', 'L', 'SV',
       'WHIP', 'ERA', 'WAR', 'TVS'],
      dtype='object')

#### Agentes libres

No se conservará el equipo al que es contratado el agente libre puesto que esta información también la contiene la base de datos que facilita más el tratamiento _ETL_.

In [19]:
for year in range(0,period):
    df_free_agents_copy[year] = df_free_agents_copy[year].rename(columns = {'Player':'Jugador',
                                'Status':'Status_agente_libre', 'Team From':'Equipo_anterior',
                                'Value':'Valor_contrato', 'AAV':'Valor_promedio_contrato',
                                'YRS':'Anios_de_contrato'})
    
    free_agents_aux_1 = df_free_agents_copy[year]['Valor_contrato'].str.replace("$","")
    free_agents_aux_2 = free_agents_aux_1.str.replace(",","")
    free_agents_aux_3 = df_free_agents_copy[year]['Valor_promedio_contrato'].str.replace("$","")
    free_agents_aux_4 = free_agents_aux_3.str.replace(",","")
    df_free_agents_copy[year]['Valor_contrato'] = free_agents_aux_2
    df_free_agents_copy[year]['Valor_promedio_contrato'] = free_agents_aux_4
    
    df_free_agents_copy[year]['Valor_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_contrato'])
    df_free_agents_copy[year]['Valor_promedio_contrato'] = pd.to_numeric(df_free_agents_copy[year]['Valor_promedio_contrato'])

Observemos las dimensiones de las bases de datos como referencia

In [20]:
for year in range(0,period):
    print(df_free_agents_copy[year].shape)

(1, 6)
(108, 6)
(213, 6)
(208, 6)
(221, 6)
(241, 6)
(100, 6)
(98, 6)
(105, 6)
(118, 6)
(141, 6)
(137, 6)


También el tipo de datos que contiene cada columna

In [21]:
df_free_agents_copy[6].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Jugador                  100 non-null    object
 1   Status_agente_libre      100 non-null    object
 2   Equipo_anterior          100 non-null    object
 3   Anios_de_contrato        100 non-null    int64 
 4   Valor_contrato           100 non-null    int64 
 5   Valor_promedio_contrato  100 non-null    int64 
dtypes: int64(3), object(3)
memory usage: 4.8+ KB


#### Salarios

Como los salarios irán con las bases de datos de los _hitters_ y _pitchers_ es que se hará su proceso _ETL_ antes.

In [22]:
for year in range(0,period):
    # Cambio de nombres
    df_salary_copy[year] = df_salary_copy[year].rename(columns = {'Player':'Jugador',
                            'BaseSalary':'Sueldo_base', 'SigningBonus':'Bono_por_firma',
                            'Payroll Salary':'Sueldo_regular', 'Adj Salary':'Sueldo_ajustado',
                            'CONT YR':'Anios_de_contrato', 'CONT VALUE':'Valor_del_contrato',
                            'Earnings':'Ganancias', 'FA Year':'Anio_de_agente_libre',
                            'Sign Age':'Edad_al_firmar', 'Age':'Edad', 'Weight':'Peso',
                            'Height':'Altura', 'Year':'Anio', 'Pos':'Posicion',
                            'Salary%':'Sueldo_porcentual', 'Cash':'Pago_efectivo',
                            'AAV':'Valor_contrato_promedio', 'Team':'Acronimo'})
    
    # Tranformando al tipo de dato apropiado
    salary_aux_1 = df_salary_copy[year]['Sueldo_base'].str.replace("$","")
    salary_aux_2 = salary_aux_1.str.replace(",","")
    df_salary_copy[year]['Sueldo_base'] = salary_aux_2
    df_salary_copy[year]['Sueldo_base'] = pd.to_numeric(df_salary_copy[year]['Sueldo_base'])
    
    salary_aux_3 = df_salary_copy[year]['Sueldo_regular'].str.replace("$","")
    salary_aux_4 = salary_aux_3.str.replace(",","")
    df_salary_copy[year]['Sueldo_regular'] = salary_aux_4
    df_salary_copy[year]['Sueldo_regular'] = pd.to_numeric(df_salary_copy[year]['Sueldo_regular'])
    
    salary_aux_5 = df_salary_copy[year]['Sueldo_ajustado'].str.replace("$","")
    salary_aux_6 = salary_aux_5.str.replace(",","")
    df_salary_copy[year]['Sueldo_ajustado'] = salary_aux_6
    df_salary_copy[year]['Sueldo_ajustado'] = pd.to_numeric(df_salary_copy[year]['Sueldo_ajustado'])
    
    salary_aux_7 = df_salary_copy[year]['Valor_del_contrato'].str.replace("$","")
    salary_aux_8 = salary_aux_7.str.replace(",","")
    df_salary_copy[year]['Valor_del_contrato'] = salary_aux_8
    df_salary_copy[year]['Valor_del_contrato'] = pd.to_numeric(df_salary_copy[year]['Valor_del_contrato'])
    
    salary_aux_9 = df_salary_copy[year]['Bono_por_firma'].str.replace("$","")
    salary_aux_10 = salary_aux_9.str.replace(",","")
    df_salary_copy[year]['Bono_por_firma'] = salary_aux_10
    df_salary_copy[year]['Bono_por_firma'] = pd.to_numeric(df_salary_copy[year]['Bono_por_firma'])
    
    salary_aux_11 = df_salary_copy[year]['Ganancias'].str.replace("$","")
    salary_aux_12 = salary_aux_11.str.replace(",","")
    df_salary_copy[year]['Ganancias'] = salary_aux_12
    df_salary_copy[year]['Ganancias'] = pd.to_numeric(df_salary_copy[year]['Ganancias'])
    
    salary_aux_13 = df_salary_copy[year]['Pago_efectivo'].str.replace("$","")
    salary_aux_14 = salary_aux_13.str.replace(",","")
    df_salary_copy[year]['Pago_efectivo'] = salary_aux_14
    df_salary_copy[year]['Pago_efectivo'] = pd.to_numeric(df_salary_copy[year]['Pago_efectivo'])
    
    salary_aux_15 = df_salary_copy[year]['Valor_contrato_promedio'].str.replace("$","")
    salary_aux_16 = salary_aux_15.str.replace(",","")
    df_salary_copy[year]['Valor_contrato_promedio'] = salary_aux_16
    df_salary_copy[year]['Valor_contrato_promedio'] = pd.to_numeric(df_salary_copy[year]['Valor_contrato_promedio'])
    
    salary_aux_17 = df_salary_copy[year]['Altura'].str.replace("\"","")
    salary_aux_18 = salary_aux_17.str.replace("'","")
    df_salary_copy[year]['Altura'] = salary_aux_18
    df_salary_copy[year]['Altura'] = pd.to_numeric(df_salary_copy[year]['Altura'])/10
    # SUstitullamos los xeros
    height_mean = df_salary_copy[year]['Altura'].mean(skipna=True)
    df_salary_copy[year]['Altura'] = df_salary_copy[year].Altura.mask(df_salary_copy[year].Altura == 0, height_mean)
    
    df_salary_copy[year]['Anio_de_agente_libre'] = pd.to_numeric(df_salary_copy[year]['Anio_de_agente_libre'])
    df_salary_copy[year]['Anios_de_contrato'] = pd.to_numeric(df_salary_copy[year]['Anios_de_contrato'])
    df_salary_copy[year]['Edad'] = pd.to_numeric(df_salary_copy[year]['Edad'])

Por algunas particularidades de la base de datos, las columna que contiene la edad al firmar se tratará por separado aprovechando que la mayoría de los datos incorrectos tienen una longitud mayor a dos.

In [23]:
for year in range (0,period):
    df_salary_copy[year]['Edad_al_firmar'] = df_salary_copy[year]['Edad_al_firmar'].map(str)

    for edad in range(0,df_salary_copy[year].shape[0]):
        # String es mayor que 0:
        if len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) == 2:
            df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])
            
        # String es menor o igual que 0:
        elif len(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != 2:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)    
    
        # Entero  menor a 0:
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            # Si la columna de la edad contiene datos correctos
            if df_salary_copy[year]['Edad'].iloc[edad] > 0:
                if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                    ag_year = year + 2011 + 1
                else:
                    ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
                # Get first year of contract
                ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
                # Años desde el el año inicial
                dif_years = year + 2011 - ini_year
                # Edad al firmar:
                sign_age = df_salary_copy[year]['Edad'].iloc[edad] - dif_years
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(sign_age)
                
            # Si la columna de edad no contiene un dato coherente
            else:
                # Cambio de dato:
                df_salary_copy[year].iloc[edad, df_salary_copy[year].columns.get_loc('Edad_al_firmar')] = pd.to_numeric(18)
         
    # Transformemos los datos a enteros
    df_salary_copy[year]['Edad_al_firmar'] = pd.to_numeric(df_salary_copy[year]['Edad_al_firmar'])

Podemos verificar si se limpiaron adecuadamente las celdas de la columna de edades al firmar. Esto, al filtrar los datos que sean distintos a enteros y al observar si se pudo transformar toda la columna al tipo entero.

In [24]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) != np.int64:
            print(type(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]))

In [25]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
        if df_salary_copy[year]['Edad_al_firmar'].iloc[edad] < 0:
            print(df_salary_copy[year]['Edad_al_firmar'].iloc[edad])

In [26]:
#for year in range(0,period):
#    print(type(df_salary_copy[year][['Edad_al_firmar']].info()))

In [27]:
#for year in range(0,period):
#    print(year)
#    for edad in range(0,df_salary_copy[year]['Edad_al_firmar'].shape[0]):
#        print(str(df_salary_copy[year]['Edad_al_firmar'].iloc[edad]) + ' ' + str(edad))

Por otro lado, falta corregir las entradas de las columnas de las edades que tengan valores menores a cero. Esto se hará de acuerdo al resto de columnas

In [28]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(df_salary_copy[year]['Edad'].iloc[edad])

0
-9
1
-9
2
-7


In [29]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year].shape[0]):
        # Condición para imputar:
        if df_salary_copy[year]['Edad'].iloc[edad] <= 0:
            # Si no se indica si tendrá año de agencia libre:
            if df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad] == 0:
                        ag_year = year + 2011 + 1
            # Si tendrá año de agencia libre
            else:
                ag_year = df_salary_copy[year]['Anio_de_agente_libre'].iloc[edad]
            # Get first year of contract
            ini_year = ag_year - df_salary_copy[year]['Anios_de_contrato'].iloc[edad]
            # Años desde el el año inicial
            dif_years = year + 2011 - ini_year
            # Edad en la temporada:
            seasson_age = df_salary_copy[year]['Edad_al_firmar'].iloc[edad] + dif_years
            # Asignación
            df_salary_copy[year]['Edad'].iloc[edad] = seasson_age

Comprobemos que no hay ninguna edad negativa

In [30]:
for year in range(0,period):
    for edad in range(0,df_salary_copy[year]['Edad'].shape[0]):
        if df_salary_copy[year]['Edad'].iloc[edad] < 0:
            print(year)
            print(str(df_salary_copy[year]['Edad'].iloc[edad]) + ' ' + str(edad))

Con la imputación de datos ya se puede crear la columna que contiene la antiguedad del agente libre bajo el contrato

In [31]:
for year in range(0,period):
    df_salary_copy[year]['Antiguedad'] = df_salary_copy[year]['Edad'] - df_salary_copy[year]['Edad_al_firmar']

Por último, convirtamos la columna del año a string para que se entienda como una categoría y no una variable numérica

In [32]:
#for year in range(0,period):
#    df_salary_copy[year]['Anio'] = df_salary_copy[year]['Anio'].map(str)

In [33]:
df_salary_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1045 entries, 0 to 1044
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1045 non-null   object 
 1   Anio                     1045 non-null   int64  
 2   Posicion                 1045 non-null   object 
 3   Acronimo                 1045 non-null   object 
 4   Sueldo_base              1045 non-null   int64  
 5   Bono_por_firma           1045 non-null   int64  
 6   Sueldo_regular           1045 non-null   int64  
 7   Sueldo_ajustado          1045 non-null   int64  
 8   Sueldo_porcentual        1045 non-null   float64
 9   Pago_efectivo            1045 non-null   int64  
 10  Valor_contrato_promedio  1045 non-null   int64  
 11  Anios_de_contrato        1045 non-null   int64  
 12  Valor_del_contrato       1045 non-null   int64  
 13  Ganancias                1045 non-null   int64  
 14  Anio_de_agente_libre    

#### Hitters

In [34]:
for year in range(0,period):
    # Cambio de nombres
    df_hitting_copy[year] = df_hitting_copy[year].rename(columns = {'Player':'Jugador',
                            'GP':'Juegos', 'GP%':'Porcentaje_juegos',
                            'AB':'At-bats', 'H':'Bateos', 'GS':'Juegos_iniciados',
                            'GS%':'Porcentaje_juegos_iniciados', 'RBI':'Runs-batted-in',
                            'HR':'Home-runs', 'AVG':'Bateos_promedio',
                            '2B':'Dobles', '3B':'Triples', 'OPS':'Porcentaje_On-base-plus-slugging',
                            'SLG':'Porcentaje_slugging', 'OBP':'Porcentaje_on-base'})

In [35]:
for year in range(0,period):
    print(df_hitting_copy[year].shape)

(521, 17)
(669, 17)
(734, 17)
(778, 17)
(809, 17)
(834, 17)
(810, 17)
(825, 17)
(850, 17)
(451, 17)
(753, 17)
(496, 17)


In [36]:
df_hitting_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 834 entries, 0 to 833
Data columns (total 17 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Jugador                           834 non-null    object 
 1   Juegos                            834 non-null    int64  
 2   Porcentaje_juegos                 834 non-null    float64
 3   Juegos_iniciados                  834 non-null    int64  
 4   Porcentaje_juegos_iniciados       834 non-null    float64
 5   At-bats                           834 non-null    int64  
 6   Bateos                            834 non-null    int64  
 7   Dobles                            834 non-null    int64  
 8   Triples                           834 non-null    int64  
 9   Home-runs                         834 non-null    int64  
 10  Runs-batted-in                    834 non-null    int64  
 11  Bateos_promedio                   834 non-null    float64
 12  Porcenta

In [37]:
df_hitting_copy[5].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos', 'Dobles', 'Triples',
       'Home-runs', 'Runs-batted-in', 'Bateos_promedio', 'Porcentaje_on-base',
       'Porcentaje_slugging', 'Porcentaje_On-base-plus-slugging', 'WAR',
       'TVS'],
      dtype='object')

#### Pitchers

In [38]:
for year in range(0,period):
    # Cambio de nombres
    df_pitching_copy[year] = df_pitching_copy[year].rename(columns = {'Player':'Jugador',
                             'GP':'Juegos', 'GS':'Juegos_iniciados', 'IP':'Inning_pitched',
                             'H':'Bateos', 'R':'Carreras', 'ER':'Carreras_ganadas',
                             'BB':'Walks', 'SO':'Strike-outs', 'W':'Wins', 'L':'Losses',
                             'SV':'Saves'})

In [39]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(324, 16)
(427, 16)
(485, 16)
(537, 16)
(586, 16)
(592, 16)
(581, 16)
(644, 16)
(668, 16)
(542, 16)
(611, 16)
(598, 16)


In [40]:
df_pitching_copy[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Jugador           592 non-null    object 
 1   Juegos            592 non-null    int64  
 2   Juegos_iniciados  592 non-null    int64  
 3   Inning_pitched    592 non-null    float64
 4   Bateos            592 non-null    int64  
 5   Carreras          592 non-null    int64  
 6   Carreras_ganadas  592 non-null    int64  
 7   Walks             592 non-null    int64  
 8   Strike-outs       592 non-null    int64  
 9   Wins              592 non-null    int64  
 10  Losses            592 non-null    int64  
 11  Saves             592 non-null    int64  
 12  WHIP              592 non-null    float64
 13  ERA               592 non-null    float64
 14  WAR               587 non-null    float64
 15  TVS               592 non-null    float64
dtypes: float64(5), int64(10), object(1)
memory u

## Agregación de variables sugeridas por artículos

Las primeras variables que agregaremos son el cuadrado de todas las estadísticas deportivas, así como las siguientes variables:

- DOMINANCE = $Strike-outs/(Inning \; Pitched)$
- CONTROL = $Walks/(Inning \; Pitched)$
- COMMAND = $Strike-outs/Walks$

In [41]:
for year in range(0,period):
    df_pitching_copy[year]['Dominio'] = df_pitching_copy[year]['Strike-outs']/(df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Control'] = df_pitching_copy[year]['Walks']/(df_pitching_copy[year]['Inning_pitched'])
    df_pitching_copy[year]['Comando'] = df_pitching_copy[year]['Strike-outs']/df_pitching_copy[year]['Walks']

In [42]:
for year in range(0,period):
    print(df_pitching_copy[year].shape)

(324, 19)
(427, 19)
(485, 19)
(537, 19)
(586, 19)
(592, 19)
(581, 19)
(644, 19)
(668, 19)
(542, 19)
(611, 19)
(598, 19)


In [43]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Jugador           485 non-null    object 
 1   Juegos            485 non-null    int64  
 2   Juegos_iniciados  485 non-null    int64  
 3   Inning_pitched    485 non-null    float64
 4   Bateos            485 non-null    int64  
 5   Carreras          485 non-null    int64  
 6   Carreras_ganadas  485 non-null    int64  
 7   Walks             485 non-null    int64  
 8   Strike-outs       485 non-null    int64  
 9   Wins              485 non-null    int64  
 10  Losses            485 non-null    int64  
 11  Saves             485 non-null    int64  
 12  WHIP              485 non-null    float64
 13  ERA               485 non-null    float64
 14  WAR               435 non-null    float64
 15  TVS               485 non-null    float64
 16  Dominio           484 non-null    float64
 1

Podemos verificar qué entradas poseen valores infinitos en la base de datos

In [44]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Recordemos que el modelo dinámico supone que la compensación salarial está normalizada entre 0 y 1. Por lo que se normalizarán los logaritmos de la compensación, así como las compensacioes en nómina/regulares.

In [45]:
# Define the columns to normalize
comp_to_normalize = ['Sueldo_base', 'Sueldo_ajustado',
                     'Sueldo_regular']

In [46]:
# For cyle for all years:
for year in range(0,period):
    # For cycle for all compensation variables
    for comp in comp_to_normalize:
        new_name = comp + '_norm'
        df_salary_copy[year][new_name] = df_salary_copy[year][comp]

In [47]:
# Define the columns to normalize
normal_compensation = ['Sueldo_base_norm', 'Sueldo_ajustado_norm',
                       'Sueldo_regular_norm']

In [48]:
# For cycle to normalize
for year in range(0,period):
    # Create a MinMaxScaler object
    scaler = MinMaxScaler()

    # Normalize the columns using the fit_transform() method
    df_salary_copy[year][normal_compensation] = scaler.fit_transform(df_salary_copy[year][normal_compensation])

Siguiendo la sugerencia de algunos artículos, obtengamos el logaritmo de los salarios

In [49]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_base'] = np.log(df_salary_copy[year]['Sueldo_base'])
    df_salary_copy[year]['ln_Sueldo_ajustado'] = np.log(df_salary_copy[year]['Sueldo_ajustado'])
    df_salary_copy[year]['ln_Sueldo_regular'] = np.log(df_salary_copy[year]['Sueldo_regular'])

In [50]:
df_salary_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013 entries, 0 to 1012
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1013 non-null   object 
 1   Anio                     1013 non-null   int64  
 2   Posicion                 1013 non-null   object 
 3   Acronimo                 1013 non-null   object 
 4   Sueldo_base              1013 non-null   int64  
 5   Bono_por_firma           1013 non-null   int64  
 6   Sueldo_regular           1013 non-null   int64  
 7   Sueldo_ajustado          1013 non-null   int64  
 8   Sueldo_porcentual        1013 non-null   float64
 9   Pago_efectivo            1013 non-null   int64  
 10  Valor_contrato_promedio  1013 non-null   int64  
 11  Anios_de_contrato        1013 non-null   int64  
 12  Valor_del_contrato       1013 non-null   int64  
 13  Ganancias                1013 non-null   int64  
 14  Anio_de_agente_libre    

Debido a que hay columnas con datos tipo _Nan_ o _NULL_, optaremos por imputarlos.

Mientras que los valores infinitos generados por las nuevas variables se sustituirán dependediendo del caso:

- 0/0: 0
- num/0: Máximo de la columna correspondientefijarán

In [51]:
for year in range(0,period):
    # Salaries
    mean_hgt = df_salary_copy[year].loc[df_salary_copy[year]['Altura'] > 4.9].Altura.mean()
    mean_wgh = df_salary_copy[year].loc[df_salary_copy[year]['Peso'] > 0].Peso.mean()
    df_salary_copy[year]['Altura'].fillna(value = mean_hgt, inplace = True)
    df_salary_copy[year]['Altura'].mask(df_salary_copy[year]['Altura'] <= 4.9, mean_hgt, inplace = True)
    df_salary_copy[year]['Peso'].fillna(value = mean_wgh, inplace = True)
    df_salary_copy[year]['Peso'].mask(df_salary_copy[year]['Peso'] <= 0, mean_wgh, inplace = True)
    
    # Pitchers
    mean_war = df_pitching_copy[year].loc[df_pitching_copy[year]['WAR'] > 0].WAR.mean()
    mean_dom = df_pitching_copy[year].loc[df_pitching_copy[year]['Dominio'] > 0].Dominio.mean()
    mean_con = df_pitching_copy[year].loc[df_pitching_copy[year]['Control'] > 0].Control.mean()
    mean_com = df_pitching_copy[year].loc[df_pitching_copy[year]['Comando'] > 0].Comando.mean()
    df_pitching_copy[year]['WAR'].fillna(value = mean_war, inplace = True)
    df_pitching_copy[year]['WAR'].mask(df_pitching_copy[year]['WAR'] <= 0, mean_war, inplace = True)
    df_pitching_copy[year]['Dominio'].fillna(value = mean_dom, inplace = True)
    df_pitching_copy[year]['Dominio'].mask(df_pitching_copy[year]['Dominio'] <= 0, mean_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = mean_con, inplace = True)
    df_pitching_copy[year]['Control'].mask(df_pitching_copy[year]['Control'] <= 0, mean_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = mean_com, inplace = True)
    df_pitching_copy[year]['Comando'].mask(df_pitching_copy[year]['Comando'] <= 0, mean_com, inplace = True)
    
    # Hitters
    mean_war = df_hitting_copy[year].loc[df_hitting_copy[year]['WAR'] > 0].WAR.mean()
    df_hitting_copy[year]['WAR'].fillna(value = mean_war, inplace = True)
    df_hitting_copy[year]['WAR'].mask(df_hitting_copy[year]['WAR'] <= 0, mean_war, inplace = True)

In [52]:
for year in range(0,period):   
    # Condiciones
    con_dom_1 = df_pitching_copy[year]['Strike-outs'] == 0
    con_con_1 = df_pitching_copy[year]['Walks'] == 0
    con_com_1 = df_pitching_copy[year]['Strike-outs'] == 0
                 
    # Imputación caso 0/0
    df_pitching_copy[year].loc[con_dom_1, "Dominio"] = 0
    df_pitching_copy[year].loc[con_con_1, "Control"] = 0
    df_pitching_copy[year].loc[con_com_1, "Comando"] = 0

In [53]:
for year in range(0,period):   
    # Máximos
    max_dom = df_pitching_copy[year]['Strike-outs'].max()/9
    max_con = df_pitching_copy[year]['Walks'].max()/9
    max_com = df_pitching_copy[year]['Strike-outs'].max()
    
    # Cambianfdo infinitos a NaNs
    df_pitching_copy[year]["Dominio"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Control"].replace([np.inf, -np.inf], np.nan, inplace = True)
    df_pitching_copy[year]["Comando"].replace([np.inf, -np.inf], np.nan, inplace = True)
    
    # Imputación
    df_pitching_copy[year]['Dominio'].fillna(value = max_dom, inplace = True)
    df_pitching_copy[year]['Control'].fillna(value = max_con, inplace = True)
    df_pitching_copy[year]['Comando'].fillna(value = max_com, inplace = True)

Verifiquemos que ya no haya problemas con valores infinitos

In [54]:
"""
for year in range(0,period):
    print(str(2011 + year))
    for name in df_pitching_copy[year].columns:
        print(name)
        if type(name) != str:
            for element in range(0,len(df_pitching_copy[year][name])):
                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:
                    print(str(element) +  '  ' + str(df_pitching_copy[year][name].iloc[element]))
    print("")
"""

'\nfor year in range(0,period):\n    print(str(2011 + year))\n    for name in df_pitching_copy[year].columns:\n        print(name)\n        if type(name) != str:\n            for element in range(0,len(df_pitching_copy[year][name])):\n                if math.isinf(df_pitching_copy[year][name].iloc[element]) == True:\n                    print(str(element) +  \'  \' + str(df_pitching_copy[year][name].iloc[element]))\n    print("")\n'

Así mismo, contemos los valores *NaN* que queden presentes

In [55]:
for year in range(0,period):
    print('Año: ' + str(2011 + year))
    print('Hitters:')
    df_hitting_copy[year].isna().sum()
    print('Pitchers:')
    df_pitching_copy[year].isna().sum()
    print('Free agents:')
    df_free_agents_copy[year].isna().sum()
    print('Salaries:')
    df_salary_copy[year].isna().sum()
    print("")

Año: 2011
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2012
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2013
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2014
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2015
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2016
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2017
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2018
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2019
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2020
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2021
Hitters:
Pitchers:
Free agents:
Salaries:

Año: 2022
Hitters:
Pitchers:
Free agents:
Salaries:



Ahora, repitamos este proceso para la base de datos de los salarios.

In [56]:
salary_names = ['ln_Sueldo_ajustado', 'ln_Sueldo_base', 'ln_Sueldo_regular']

In [57]:
for name in salary_names:
    print(name)
    
    for year in range(0,period):
        print(str(2011 + year))
        for element in range(0,len(df_salary_copy[year][name])):
            if df_salary_copy[year][name].iloc[element] <= 0:
                print(str(element) +  '  ' + str(df_salary_copy[year][name].iloc[element]))
        print("")

ln_Sueldo_ajustado
2011

2012

2013

2014

2015

2016

2017

2018
72  -inf
188  -inf

2019

2020

2021
6  -inf
55  -inf
166  -inf
274  -inf

2022
193  -inf

ln_Sueldo_base
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022

ln_Sueldo_regular
2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



Al inspecsionar los errores nos damos cuenta que solo se desconocen los salarios ajustados y los fijaron a $0$. Usaremos el logaritmo base 10 del salario regular para sustituir dicho valor.

In [58]:
for year in range(0,period):
    df_salary_copy[year]['ln_Sueldo_ajustado'].mask(df_salary_copy[year]['ln_Sueldo_ajustado'] < 0,
                                                    df_salary_copy[year]['ln_Sueldo_regular'],
                                                    inplace = True)

In [59]:
for year in range(0,period):
    print(str(2011 + year))
    for element in range(0,len(df_salary_copy[year]['ln_Sueldo_ajustado'])):
        if df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element] <= 0:
            print(str(element) +  '  ' + str(df_salary_copy[year]['ln_Sueldo_ajustado'].iloc[element]))
    print("")

2011

2012

2013

2014

2015

2016

2017

2018

2019

2020

2021

2022



In [60]:
for year in range(0,period):
    print("Ajustado: " + str(df_salary_copy[year]['ln_Sueldo_ajustado'].mean())
          + 'n'
          + 'Regular: ' + str(df_salary_copy[year]['ln_Sueldo_regular'].mean()))

Ajustado: 13.785784770849762nRegular: 13.888874386883515
Ajustado: 13.81181498978195nRegular: 13.90039782504837
Ajustado: 13.54819591748778nRegular: 13.978975725734408
Ajustado: 13.66411021920866nRegular: 14.080738990191257
Ajustado: 13.49415866539551nRegular: 13.99344595993313
Ajustado: 13.478992231977141nRegular: 14.0261405412466
Ajustado: 13.575363447521962nRegular: 14.079742661574734
Ajustado: 13.562311948859815nRegular: 14.080756222584538
Ajustado: 13.521364643693584nRegular: 14.066559580453122
Ajustado: 13.029458522392627nRegular: 14.216780831796356
Ajustado: 13.70981799580226nRegular: 14.198804602895482
Ajustado: 13.924314685157658nRegular: 14.351734783233747


En efecto, ya no hay valores _NaN_ o _infinitos_.

Con el objetivo de hacer más eficiente la creación de las variables al cuadrado, lo haremos extrayendo el índice de las columnas de interés

In [61]:
df_hitting_copy[0].columns

Index(['Jugador', 'Juegos', 'Porcentaje_juegos', 'Juegos_iniciados',
       'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos', 'Dobles', 'Triples',
       'Home-runs', 'Runs-batted-in', 'Bateos_promedio', 'Porcentaje_on-base',
       'Porcentaje_slugging', 'Porcentaje_On-base-plus-slugging', 'WAR',
       'TVS'],
      dtype='object')

In [62]:
df_pitching_copy[1].columns

Index(['Jugador', 'Juegos', 'Juegos_iniciados', 'Inning_pitched', 'Bateos',
       'Carreras', 'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins',
       'Losses', 'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
       'Comando'],
      dtype='object')

In [63]:
def get_col_indices(df, names):
    return df.columns.get_indexer(names)

In [64]:
hitting_names = ['Juegos_iniciados', 'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos',
                  'Dobles', 'Triples', 'Home-runs', 'Runs-batted-in', 'Bateos_promedio',
                  'Porcentaje_on-base', 'Porcentaje_slugging', 'TVS',
                  'Porcentaje_On-base-plus-slugging', 'WAR']	
pitching_names = ['Inning_pitched', 'Bateos', 'Carreras',
                  'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins', 'Losses',
                  'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
                  'Comando']

Con el objetivo de simplificar el código, verifiquemos si todos los índices en cada base de datos son los mismos

In [65]:
print('Hitters:')
for year in range(0,period):
    print(get_col_indices(df_hitting_copy[year], hitting_names))
    
print('Pitchers:')
for year in range(0,period):
    print(get_col_indices(df_pitching_copy[year], pitching_names))

Hitters:
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
[ 3  4  5  6  7  8  9 10 11 12 13 16 14 15]
Pitchers:
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
[ 3

In [66]:
hitting_indexes = list(get_col_indices(df_hitting_copy[0], hitting_names))
pitching_indexes = list(get_col_indices(df_pitching_copy[0], pitching_names))

In [67]:
for year in range(0,period):
    # Hitters:
    for hitter_name in hitting_indexes:
        df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name] + '_2'] = np.power(df_hitting_copy[year][df_hitting_copy[year].columns[hitter_name]], 2)
    # Pitchers:
    for pitcher_name in pitching_indexes:
        df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name] + '_2'] = np.power(df_pitching_copy[year][df_pitching_copy[year].columns[pitcher_name]], 2)

Apreciemos el resultado final

In [68]:
df_hitting_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734 entries, 0 to 733
Data columns (total 31 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             734 non-null    object 
 1   Juegos                              734 non-null    int64  
 2   Porcentaje_juegos                   734 non-null    float64
 3   Juegos_iniciados                    734 non-null    int64  
 4   Porcentaje_juegos_iniciados         734 non-null    float64
 5   At-bats                             734 non-null    int64  
 6   Bateos                              734 non-null    int64  
 7   Dobles                              734 non-null    int64  
 8   Triples                             734 non-null    int64  
 9   Home-runs                           734 non-null    int64  
 10  Runs-batted-in                      734 non-null    int64  
 11  Bateos_promedio                     734 non-n

In [69]:
df_pitching_copy[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Jugador             485 non-null    object 
 1   Juegos              485 non-null    int64  
 2   Juegos_iniciados    485 non-null    int64  
 3   Inning_pitched      485 non-null    float64
 4   Bateos              485 non-null    int64  
 5   Carreras            485 non-null    int64  
 6   Carreras_ganadas    485 non-null    int64  
 7   Walks               485 non-null    int64  
 8   Strike-outs         485 non-null    int64  
 9   Wins                485 non-null    int64  
 10  Losses              485 non-null    int64  
 11  Saves               485 non-null    int64  
 12  WHIP                485 non-null    float64
 13  ERA                 485 non-null    float64
 14  WAR                 485 non-null    float64
 15  TVS                 485 non-null    float64
 16  Dominio 

## Unión de las bases de datos
### Datos agregados por equipo

Solo resta añadir los datos relevantes al equipo al que pertenece cada jugador considerando la base de datos de la cantidad de equipos por estado

In [70]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
dtypes: float64(1), int64(7), object(2)
memory usage: 2.3+ KB


In [71]:
acronym_state.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Estado               30 non-null     object
 1   Cantidad de equipos  30 non-null     int64 
 2   Equipo               30 non-null     object
 3   Acronimo             30 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.2+ KB


In [72]:
for year in range(0,period):
    df_teams_copy[year] = pd.merge(df_teams_copy[year], acronym_state, on = ['Equipo','Acronimo'])

In [73]:
df_teams_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28 entries, 0 to 27
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   28 non-null     object 
 1   Cantidad_agentes_libres  28 non-null     int64  
 2   Valor_contrato_total     28 non-null     int64  
 3   Acronimo                 28 non-null     object 
 4   Victorias                28 non-null     int64  
 5   Juegos totales           28 non-null     int64  
 6   Playoffs                 28 non-null     int64  
 7   Pennants won             28 non-null     int64  
 8   WS ganadas               28 non-null     int64  
 9   Promedio_victorias       28 non-null     float64
 10  Estado                   28 non-null     object 
 11  Cantidad de equipos      28 non-null     int64  
dtypes: float64(1), int64(8), object(3)
memory usage: 2.8+ KB


In [74]:
df_salary_copy[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  1036 non-null   object 
 1   Anio                     1036 non-null   int64  
 2   Posicion                 1036 non-null   object 
 3   Acronimo                 1036 non-null   object 
 4   Sueldo_base              1036 non-null   int64  
 5   Bono_por_firma           1036 non-null   int64  
 6   Sueldo_regular           1036 non-null   int64  
 7   Sueldo_ajustado          1036 non-null   int64  
 8   Sueldo_porcentual        1036 non-null   float64
 9   Pago_efectivo            1036 non-null   int64  
 10  Valor_contrato_promedio  1036 non-null   int64  
 11  Anios_de_contrato        1036 non-null   int64  
 12  Valor_del_contrato       1036 non-null   int64  
 13  Ganancias                1036 non-null   int64  
 14  Anio_de_agente_libre    

Ahora, unamos las bases de datos sobre los equipos a las bases de datos de los salarios

In [75]:
for year in range(0,period):
    df_salary_copy[year] = pd.merge(df_teams_copy[year], df_salary_copy[year], on = 'Acronimo')

In [76]:
df_salary_copy[0].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Equipo                   40 non-null     object 
 1   Cantidad_agentes_libres  40 non-null     int64  
 2   Valor_contrato_total     40 non-null     int64  
 3   Acronimo                 40 non-null     object 
 4   Victorias                40 non-null     int64  
 5   Juegos totales           40 non-null     int64  
 6   Playoffs                 40 non-null     int64  
 7   Pennants won             40 non-null     int64  
 8   WS ganadas               40 non-null     int64  
 9   Promedio_victorias       40 non-null     float64
 10  Estado                   40 non-null     object 
 11  Cantidad de equipos      40 non-null     int64  
 12  Jugador                  40 non-null     object 
 13  Anio                     40 non-null     int64  
 14  Posicion                 40 

Debido a que la mayoría de los jugadores juega tanto en la ofensiva como la defensiva es que tenemos que borrar los duplicados de la columna de la posición.

In [77]:
for year in range(0,period):
    df_hitting_copy[year] = pd.merge(df_hitting_copy[year], df_salary_copy[year], on = 'Jugador')
    df_pitching_copy[year] = pd.merge(df_pitching_copy[year], df_salary_copy[year], on = 'Jugador')

In [78]:
for year in range(0,period):
    df_pitching_copy[year]['Porcentaje_juegos'] = df_pitching_copy[year]['Juegos']/df_pitching_copy[year]['Juegos totales']

In [79]:
df_hitting_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 667 entries, 0 to 666
Data columns (total 67 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Jugador                             667 non-null    object 
 1   Juegos                              667 non-null    int64  
 2   Porcentaje_juegos                   667 non-null    float64
 3   Juegos_iniciados                    667 non-null    int64  
 4   Porcentaje_juegos_iniciados         667 non-null    float64
 5   At-bats                             667 non-null    int64  
 6   Bateos                              667 non-null    int64  
 7   Dobles                              667 non-null    int64  
 8   Triples                             667 non-null    int64  
 9   Home-runs                           667 non-null    int64  
 10  Runs-batted-in                      667 non-null    int64  
 11  Bateos_promedio                     667 non-n

In [80]:
df_pitching_copy[3].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497 entries, 0 to 496
Data columns (total 72 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Jugador                  497 non-null    object 
 1   Juegos                   497 non-null    int64  
 2   Juegos_iniciados         497 non-null    int64  
 3   Inning_pitched           497 non-null    float64
 4   Bateos                   497 non-null    int64  
 5   Carreras                 497 non-null    int64  
 6   Carreras_ganadas         497 non-null    int64  
 7   Walks                    497 non-null    int64  
 8   Strike-outs              497 non-null    int64  
 9   Wins                     497 non-null    int64  
 10  Losses                   497 non-null    int64  
 11  Saves                    497 non-null    int64  
 12  WHIP                     497 non-null    float64
 13  ERA                      497 non-null    float64
 14  WAR                      4

Para facilitar la observación de las trnasformaciones de manera más eficiente, ordenemos alfabéticamente la base de datos de acuerdo al nombre de las columnas.

In [81]:
for year in range(0,period):
    # Ordenando alfabéticamente
    df_salary_copy[year].sort_index(axis = 1, inplace = True)
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)
    df_free_agents_copy[year].sort_index(axis = 1, inplace = True)
    
    # Reiniciando los índices
    df_salary_copy[year].reset_index(drop = True, inplace = True)
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)
    df_free_agents_copy[year].reset_index(drop = True, inplace = True)

## Variables del periodo t-1

Lo que haremos será un *merge* de las bases de datos del año $t$ con el año $t-1$ sobre los jugadores. La razón de esto es que solo nos interesan los jugadores que han sido agentes libres por más de un año.

Si la primera base de datos es del año 2011, entonces tendremos que empezar en el año 2012. Creemos los dataframes que contendrán los datos para el modelo. Para que no se sobrepongan todos los periodos, crearemos dataframes auxiliares para guardar los nuevos datos

In [82]:
hitting_merge = ['Juegos_iniciados', 'Porcentaje_juegos_iniciados', 'At-bats', 'Bateos',
                 'Dobles', 'Triples', 'Home-runs', 'Runs-batted-in', 'Bateos_promedio',
                 'Porcentaje_on-base', 'Porcentaje_slugging', 'TVS',
                 'Porcentaje_On-base-plus-slugging', 'WAR',
                 'Juegos_iniciados_2', 'Porcentaje_juegos_iniciados_2', 'At-bats_2', 'Bateos_2',
                 'Dobles_2', 'Triples_2', 'Home-runs_2', 'Runs-batted-in_2', 'Bateos_promedio_2',
                 'Porcentaje_on-base_2', 'Porcentaje_slugging_2', 'TVS_2',
                 'Porcentaje_On-base-plus-slugging_2', 'WAR_2']	
pitching_merge = ['Inning_pitched', 'Bateos_en_contra', 'Carreras_en_contra',
                  'Carreras_ganadas', 'Walks', 'Strike-outs', 'Wins', 'Losses',
                  'Saves', 'WHIP', 'ERA', 'WAR', 'TVS', 'Dominio', 'Control',
                  'Comando',
                  'Inning_pitched_2', 'Bateos_2', 'Carreras_2',
                  'Carreras_ganadas_2', 'Walks_2', 'Strike-outs_2', 'Wins_2', 'Losses_2',
                  'Saves_2', 'WHIP_2', 'ERA_2', 'WAR_2', 'TVS_2', 'Dominio_2', 'Control_2',
                  'Comando_2']

In [83]:
df_hitters_copy = [None]*period
df_pitchers_copy = [None]*period

In [84]:
for year in range(0,period):
    df_hitters_copy[year] = df_hitting_copy[year].copy()
    df_pitchers_copy[year] = df_pitching_copy[year].copy()

In [85]:
for year in range(1,period):    
    df_hitting_copy[year] = pd.merge(df_hitters_copy[year], df_hitters_copy[year-1], on = 'Jugador')
    df_pitching_copy[year] = pd.merge(df_pitchers_copy[year], df_pitchers_copy[year-1], on = 'Jugador')

A continuación se verifica que la cantidad de columnas sea la misma, salvo por el primer periodo

In [86]:
"""for name in df_pitching_copy[11].columns:
    print(name)"""

'for name in df_pitching_copy[11].columns:\n    print(name)'

In [87]:
for year in range(0,period):
    print(df_hitting_copy[year].columns.shape)
    
for year in range(0,period):    
    print(df_pitching_copy[year].columns.shape)

(67,)
(133,)
(133,)
(133,)
(133,)
(133,)
(133,)
(133,)
(133,)
(133,)
(133,)
(133,)
(72,)
(143,)
(143,)
(143,)
(143,)
(143,)
(143,)
(143,)
(143,)
(143,)
(143,)
(143,)


In [88]:
for year in range(1,period):       
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('_x', '_t')
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('_y', '_t_1')
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('-', '_')
    df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace(' ', '_')
    df_pitching_copy[year].drop(['ln_Sueldo_base_t_1', 'ln_Sueldo_ajustado_t_1', 'ln_Sueldo_regular_t_1'],
                           axis = 1, inplace = True)
    df_pitching_copy[year] = df_pitching_copy[year].sort_values(by = 'Jugador', ascending = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)
    
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('_x', '_t')
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('_y', '_t_1')
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('-', '_')
    df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace(' ', '_')
    df_hitting_copy[year].drop(['ln_Sueldo_base_t_1', 'ln_Sueldo_ajustado_t_1', 'ln_Sueldo_regular_t_1'],
                          axis = 1, inplace = True)
    df_hitting_copy[year] = df_hitting_copy[year].sort_values(by = 'Jugador', ascending = True)
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    
    # Reordenando las columnas
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)

In [89]:
for name in df_pitching_copy[11].columns:
    print(name)

Acronimo_t
Acronimo_t_1
Altura_t
Altura_t_1
Anio_de_agente_libre_t
Anio_de_agente_libre_t_1
Anio_t
Anio_t_1
Anios_de_contrato_t
Anios_de_contrato_t_1
Antiguedad_t
Antiguedad_t_1
Bateos_2_t
Bateos_2_t_1
Bateos_t
Bateos_t_1
Bono_por_firma_t
Bono_por_firma_t_1
Cantidad_agentes_libres_t
Cantidad_agentes_libres_t_1
Cantidad_de_equipos_t
Cantidad_de_equipos_t_1
Carreras_2_t
Carreras_2_t_1
Carreras_ganadas_2_t
Carreras_ganadas_2_t_1
Carreras_ganadas_t
Carreras_ganadas_t_1
Carreras_t
Carreras_t_1
Comando_2_t
Comando_2_t_1
Comando_t
Comando_t_1
Control_2_t
Control_2_t_1
Control_t
Control_t_1
Dominio_2_t
Dominio_2_t_1
Dominio_t
Dominio_t_1
ERA_2_t
ERA_2_t_1
ERA_t
ERA_t_1
Edad_al_firmar_t
Edad_al_firmar_t_1
Edad_t
Edad_t_1
Equipo_t
Equipo_t_1
Estado_t
Estado_t_1
Ganancias_t
Ganancias_t_1
Inning_pitched_2_t
Inning_pitched_2_t_1
Inning_pitched_t
Inning_pitched_t_1
Juegos_iniciados_t
Juegos_iniciados_t_1
Juegos_t
Juegos_t_1
Juegos_totales_t
Juegos_totales_t_1
Jugador
Losses_2_t
Losses_2_t_1
Losses_t

Debido a que muchas de las variables del periodo $t_1$ pueden funcionar como controles más realistas, se optarán por dejarlas a excepción de la columna que contiene el dato del año al que hace referencia el dataframe del periodo $t_1$, es decir, la columna *Anio_t_1*. Esto se hará para *pitchers* y *hitters*. Por razones análogas, también de omitirá la columna que indica la cantidad de equipos en determinado estado ya que en el periodo de análisis es invariante.

Para facilitar la escritura del código, entenderemos la columna *Anio* como la columna *Anio_t*.

In [90]:
for year in range(1,period):
    df_pitching_copy[year].drop(['Anio_t_1', 'Estado_t_1', 'Edad_t_1'],
                           axis = 1, inplace = True)
    
    df_hitting_copy[year].drop(['Anio_t_1', 'Estado_t_1', 'Edad_t_1'],
                           axis = 1, inplace = True)
    
    # Reordenando las columnas
    df_hitting_copy[year].sort_index(axis = 1, inplace = True)
    df_pitching_copy[year].sort_index(axis = 1, inplace = True)
    
    # Reiniciando índice
    df_hitting_copy[year].reset_index(drop = True, inplace = True)
    df_pitching_copy[year].reset_index(drop = True, inplace = True)

Cambiemos el súfijo de las basses de datos del año del 2011

In [91]:
year = 0
# Reiniciando los índices
df_hitting_copy[year] = df_hitting_copy[year].add_suffix('_t')
df_pitching_copy[year] = df_pitching_copy[year].add_suffix('_t')
# Corrección de columna del jugador
df_hitting_copy[year].columns = df_hitting_copy[year].columns.str.replace('Jugador_t', 'Jugador')
df_pitching_copy[year].columns = df_pitching_copy[year].columns.str.replace('Jugador_t', 'Jugador')

In [92]:
print("Salarios")
print(df_salary_copy[year].columns)
print("\n")
print("Hitters")
print(df_hitting_copy[year].columns)
print("\n")
print("Pitchers")
print(df_pitching_copy[year].columns)
print("\n")
print("Free agents")
print(df_free_agents_copy[year].columns)
print("\n")

Salarios
Index(['Acronimo', 'Altura', 'Anio', 'Anio_de_agente_libre',
       'Anios_de_contrato', 'Antiguedad', 'Bono_por_firma',
       'Cantidad de equipos', 'Cantidad_agentes_libres', 'Edad',
       'Edad_al_firmar', 'Equipo', 'Estado', 'Ganancias', 'Juegos totales',
       'Jugador', 'Pago_efectivo', 'Pennants won', 'Peso', 'Playoffs',
       'Posicion', 'Promedio_victorias', 'Sueldo_ajustado',
       'Sueldo_ajustado_norm', 'Sueldo_base', 'Sueldo_base_norm',
       'Sueldo_porcentual', 'Sueldo_regular', 'Sueldo_regular_norm',
       'Valor_contrato_promedio', 'Valor_contrato_total', 'Valor_del_contrato',
       'Victorias', 'WS ganadas', 'ln_Sueldo_ajustado', 'ln_Sueldo_base',
       'ln_Sueldo_regular'],
      dtype='object')


Hitters
Index(['Acronimo_t', 'Altura_t', 'Anio_t', 'Anio_de_agente_libre_t',
       'Anios_de_contrato_t', 'Antiguedad_t', 'At-bats_t', 'At-bats_2_t',
       'Bateos_t', 'Bateos_2_t', 'Bateos_promedio_t', 'Bateos_promedio_2_t',
       'Bono_por_firma_t', '

## Segmentación por Agentes libres

Separaremos los pitchers y hitters en dos grupos:

- Agentes libres.
- No agentes libres.

In [93]:
for year in range(0,period):
    # Filtrando los agentes libres
    df_hitters_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                            df_hitting_copy[year], on = 'Jugador')
    df_pitchers_free_agents[year] = pd.merge(df_free_agents_copy[year],
                                             df_pitching_copy[year], on = 'Jugador')
    # FIltrando los que no son agentes libres
    df_hitters_no_free_agents[year] = df_hitting_copy[year][~df_hitting_copy[year].Jugador.isin(df_hitters_free_agents[year].Jugador)]
    df_pitchers_no_free_agents[year] = df_pitching_copy[year][~df_pitching_copy[year].Jugador.isin(df_pitchers_free_agents[year].Jugador)]
    
    # Reiniciando el índice
    df_hitters_free_agents[year] = df_hitters_free_agents[year].reindex(sorted(df_hitters_free_agents[year].columns), axis=1)
    df_pitchers_free_agents[year] = df_pitchers_free_agents[year].reindex(sorted(df_pitchers_free_agents[year].columns), axis=1)
    df_hitters_no_free_agents[year] = df_hitters_no_free_agents[year].reindex(sorted(df_hitters_no_free_agents[year].columns), axis=1)
    df_pitchers_no_free_agents[year] = df_pitchers_no_free_agents[year].reindex(sorted(df_pitchers_no_free_agents[year].columns), axis=1)    

Veamos los contenidos de las nuevas bases de datos

In [94]:
print("FA - Hitters:")
df_hitters_free_agents[9].info()
print("\n FA - Pitchers:")
df_pitchers_free_agents[9].info()
print("\n No FA - Hitters:")
df_hitters_no_free_agents[9].info()
print("\n No FA - Hitters:")
df_pitchers_no_free_agents[9].info()

FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 38 entries, 0 to 37
Columns: 132 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(46), int64(76), object(10)
memory usage: 39.5+ KB

 FA - Pitchers:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 0 to 45
Columns: 142 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(50), int64(82), object(10)
memory usage: 51.4+ KB

 No FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 241 entries, 0 to 278
Columns: 127 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(46), int64(73), object(8)
memory usage: 241.0+ KB

 No FA - Hitters:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 309 entries, 0 to 353
Columns: 137 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(50), int64(79), object(8)
memory usage: 333.1+ KB


In [95]:
print("FA - Hitters:")
for year in range(0,period):
    print(df_hitters_free_agents[year].shape)
print("\n FA - Pitchers:")
for year in range(0,period):
    print(df_pitchers_free_agents[year].shape)

FA - Hitters:
(0, 72)
(0, 132)
(116, 132)
(61, 132)
(63, 132)
(79, 132)
(45, 132)
(42, 132)
(48, 132)
(38, 132)
(29, 132)
(49, 132)

 FA - Pitchers:
(0, 77)
(2, 142)
(75, 142)
(49, 142)
(53, 142)
(63, 142)
(33, 142)
(44, 142)
(39, 142)
(46, 142)
(46, 142)
(92, 142)


Por último, para facilitar futuras aplicaciones, pasemos todos los nombres de las columnas a miníscula

In [96]:
for year in range(0,period):
    df_hitters_free_agents[year].rename(columns = str.lower)
    df_pitchers_free_agents[year].rename(columns = str.lower)
    df_hitters_no_free_agents[year].rename(columns = str.lower)
    df_pitchers_no_free_agents[year].rename(columns = str.lower)

### Etiquetas para los agentes libres

Crearemos un etiqueta para indicar si el pitcher o hitter es  un agente libre o no.

In [97]:
for year in range(0,period):
    # Condiciones
    condicion_hitter = [df_hitting_copy[year].Jugador.isin(df_free_agents_copy[year].Jugador)]
    condicion_pitcher = [df_pitching_copy[year].Jugador.isin(df_free_agents_copy[year].Jugador)]
    
    # Etiquetas
    etiquetas = ['Si']
    
    df_hitting_copy[year]['Agente_libre'] = np.select(condicion_hitter, etiquetas, default = 'No')
    df_pitching_copy[year]['Agente_libre'] = np.select(condicion_pitcher, etiquetas, default = 'No')
    
    df_hitting_copy[year] = df_hitting_copy[year].reindex(sorted(df_hitting_copy[year].columns), axis=1)
    df_pitching_copy[year] = df_pitching_copy[year].reindex(sorted(df_pitching_copy[year].columns), axis=1)

In [98]:
for year in range(0,period):
    df_hitting_copy[year].rename(columns = str.lower)
    df_pitching_copy[year].rename(columns = str.lower)

## Panel Data

Con el objetivo de contar con una base de datos en estructura panel, uniremos las bases de datos

In [99]:
# Inicialización del panel
df_panel_all_hitter = df_hitting_copy[0]
df_panel_all_pitcher = df_pitching_copy[0]

for year in range(1,period):
    # Hitter
    df_panel_all_hitter = pd.concat([df_panel_all_hitter, df_hitting_copy[year]])
    
    # Pitcher
    df_panel_all_pitcher = pd.concat([df_panel_all_pitcher, df_pitching_copy[year]])

Filtremos aquellos que no son agentes libres

In [110]:
df_panel_nofa_hitter = df_panel_all_hitter.loc[df_panel_all_hitter['Agente_libre'] == 'No'] 
df_panel_nofa_fielder = df_panel_all_pitcher.loc[df_panel_all_pitcher['Agente_libre'] == 'No'] 

In [111]:
df_panel_nofa_hitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4011 entries, 0 to 291
Columns: 142 entries, Acronimo_t to WS_ganadas_t_1
dtypes: float64(106), int64(27), object(9)
memory usage: 4.4+ MB


In [102]:
df_panel_nofa_fielder.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3351 entries, 0 to 394
Columns: 144 entries, Acronimo_t to Wins_t_1
dtypes: float64(101), int64(34), object(9)
memory usage: 3.7+ MB


In [103]:
df_panel_fa_hitter = df_panel_all_hitter.loc[df_panel_all_hitter['Agente_libre'] == 'Si'] 
df_panel_fa_fielder = df_panel_all_pitcher.loc[df_panel_all_pitcher['Agente_libre'] == 'Si'] 

In [112]:
# get the first year of observation of each player
first_year = df_panel_fa_hitter.groupby('Jugador')['Anio_t'].min()
# create a new column 'start_year' that contains the first year of observation for each player
df_panel_fa_hitter['Primer_anio'] = df_panel_fa_hitter['Jugador'].map(first_year)

# get the first year of observation of each player
first_year = df_panel_fa_fielder.groupby('Jugador')['Anio_t'].min()
# create a new column 'start_year' that contains the first year of observation for each player
df_panel_fa_fielder['Primer_anio'] = df_panel_fa_fielder['Jugador'].map(first_year)

In [113]:
df_panel_fa_fielder[['Jugador','Primer_anio']].sort_values('Jugador').head(10)

,Jugador,Primer_anio
1,A.J. Burnett,2014
0,A.J. Burnett,2014
3,Aaron Harang,2015
6,Adam Ottavino,2019
8,Adam Ottavino,2019
7,Adam Wainwright,2020
5,Adam Wainwright,2020
10,Adam Warren,2019
11,Addison Reed,2018
13,Alex Cobb,2018


In [114]:
df_panel_fa_hitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 566 entries, 4 to 284
Columns: 143 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(106), int64(28), object(9)
memory usage: 636.8+ KB


In [115]:
df_panel_fa_fielder.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 541 entries, 6 to 395
Columns: 145 entries, Acronimo_t to ln_Sueldo_regular_t
dtypes: float64(101), int64(35), object(9)
memory usage: 617.1+ KB


In [108]:
df_panel_nofa_hitter.sort_index(axis = 1, inplace = True)
df_panel_nofa_fielder.sort_index(axis = 1, inplace = True)
df_panel_fa_hitter.sort_index(axis = 1, inplace = True)
df_panel_fa_fielder.sort_index(axis = 1, inplace = True)

Creemos un dataframe que contengan solo a los jugadores en común entre las bases

In [109]:
# Bateadores:
df_hitter_change = pd.concat(df_panel_nofa_hitter,
                            df_panel_fa_hitter,
                            on = 'Jugador')

# Fildeadores:
df_fielder_change = pd.concat(df_panel_nofa_fielder,
                             df_panel_fa_fielder,
                             on = 'Jugador')

TypeError: concat() got an unexpected keyword argument 'on'

Sin embargo, el merge solo nos sirve para filtrar los nombres de los jugadores que están en ambas bases de datos, así que filtremoslos de los otros páneles

In [269]:
# Names in merge:
hitter_change_names = df_hitter_change['Jugador'].tolist()
fielder_change_names = df_fielder_change['Jugador'].tolist()

# Filtration
df_panel_nofa_hitter = df_panel_nofa_hitter[df_panel_nofa_hitter['Jugador'].isin(hitter_change_names)]
df_panel_nofa_fielder = df_panel_nofa_fielder[df_panel_nofa_fielder['Jugador'].isin(fielder_change_names)]
df_panel_fa_hitter = df_panel_fa_hitter[df_panel_fa_hitter['Jugador'].isin(hitter_change_names)]
df_panel_fa_fielder = df_panel_fa_fielder[df_panel_fa_fielder['Jugador'].isin(fielder_change_names)]

# Reiniciando índice
df_panel_nofa_hitter.reset_index(drop = True,
                                 inplace = True)
df_panel_nofa_fielder.reset_index(drop = True,
                                  inplace = True)
df_panel_fa_hitter.reset_index(drop = True,
                               inplace = True)
df_panel_fa_fielder.reset_index(drop = True,
                                inplace = True)

Agreguemos una columna auxiliar que indique el periodo relativo

In [270]:
df_panel_nofa_hitter['Tratamiento'] = 'No'
df_panel_nofa_fielder['Tratamiento'] = 'No'
df_panel_fa_hitter['Tratamiento'] = 'Si'
df_panel_fa_fielder['Tratamiento'] = 'Si'

In [271]:
df_panel_nofa_hitter['Equipo_anterior'] = 'No'
df_panel_nofa_fielder['Equipo_anterior'] = 'No'
df_panel_fa_hitter['Equipo_anterior'] = 'Si'
df_panel_fa_fielder['Equipo_anterior'] = 'Si'

Verifiquemos que las dimensiones de los renglones sean las mismas

In [272]:
df_panel_nofa_hitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1362 entries, 0 to 1361
Columns: 144 entries, Acronimo_t to Equipo_anterior
dtypes: float64(106), int64(27), object(11)
memory usage: 1.5+ MB


In [273]:
df_panel_fa_hitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 490 entries, 0 to 489
Columns: 145 entries, Acronimo_t to Equipo_anterior
dtypes: float64(106), int64(28), object(11)
memory usage: 555.2+ KB


In [274]:
df_panel_nofa_fielder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1293 entries, 0 to 1292
Columns: 146 entries, Acronimo_t to Equipo_anterior
dtypes: float64(101), int64(34), object(11)
memory usage: 1.4+ MB


In [275]:
df_panel_fa_fielder.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Columns: 147 entries, Acronimo_t to Equipo_anterior
dtypes: float64(101), int64(35), object(11)
memory usage: 569.8+ KB


Por último, concatenemos los respectivos páneles

In [276]:
# Hitter
df_hitter_change = pd.concat([df_panel_nofa_hitter,
                              df_panel_fa_hitter])

# Pitcher
df_fielder_change = pd.concat([df_panel_nofa_fielder,
                               df_panel_fa_fielder])

In [277]:
df_hitter_change.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1852 entries, 0 to 489
Columns: 145 entries, Acronimo_t to Primer_anio
dtypes: float64(107), int64(27), object(11)
memory usage: 2.1+ MB


In [278]:
df_fielder_change.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1789 entries, 0 to 495
Columns: 147 entries, Acronimo_t to Primer_anio
dtypes: float64(102), int64(34), object(11)
memory usage: 2.0+ MB


Ahora, tenemos que corregir la columna de los años creando una serie de años consecutivos a partir del primer año registrado

In [279]:
# get the first year of observation of each player
first_year = df_hitter_change.groupby('Jugador')['Anio_t'].min()

# create a new column 'start_year' that contains the first year of observation for each player
df_hitter_change['Anio_inicial'] = df_hitter_change['Jugador'].map(first_year)

# group by player_name and create a new column
df_hitter_change['Aux_Anio'] = df_hitter_change.groupby('Jugador').cumcount() + 1

# New column based on previous ones
df_hitter_change['Anio_ref'] = df_hitter_change['Anio_inicial'] + df_hitter_change['Aux_Anio'] - 1

In [280]:
# get the first year of observation of each player
first_year = df_fielder_change.groupby('Jugador')['Anio_t'].min()

# create a new column 'start_year' that contains the first year of observation for each player
df_fielder_change['Anio_inicial'] = df_fielder_change['Jugador'].map(first_year)

# group by player_name and create a new column
df_fielder_change['Aux_Anio'] = df_fielder_change.groupby('Jugador').cumcount() + 1

# New column based on previous ones
df_fielder_change['Anio_ref'] = df_fielder_change['Anio_inicial'] + df_fielder_change['Aux_Anio'] -1

In [281]:
df_hitter_change = df_hitter_change.sort_values('Jugador')

In [282]:
df_hitter_change.sort_index(axis = 1,
                            inplace = True)
df_fielder_change.sort_index(axis = 1,
                             inplace = True)
df_hitter_change.reset_index(drop = True,
                             inplace = True)
df_fielder_change.reset_index(drop = True,
                              inplace = True)

In [283]:
df_hitter_change[['Jugador','Aux_Anio','Anio_inicial','Anio_ref','Primer_anio']].sort_values(by = ['Jugador', 'Primer_anio']).head(20)

,Jugador,Aux_Anio,Anio_inicial,Anio_ref,Primer_anio
0,A.J. Burnett,3,2013,2015,2014.0
1,A.J. Burnett,4,2013,2016,2014.0
2,A.J. Burnett,2,2013,2014,NaN
3,A.J. Burnett,1,2013,2013,NaN
8,A.J. Ellis,7,2013,2019,2017.0
4,A.J. Ellis,6,2013,2018,NaN
5,A.J. Ellis,3,2013,2015,NaN
6,A.J. Ellis,2,2013,2014,NaN
7,A.J. Ellis,1,2013,2013,NaN
9,A.J. Ellis,5,2013,2017,NaN


In [284]:
df_fielder_change[['Jugador','Aux_Anio','Anio_inicial','Anio_ref','Primer_anio']].sort_values(by = ['Jugador', 'Primer_anio']).head(20)

,Jugador,Aux_Anio,Anio_inicial,Anio_ref,Primer_anio
1343,A.J. Burnett,3,2013,2015,2014.0
1386,A.J. Burnett,4,2013,2016,2014.0
19,A.J. Burnett,1,2013,2013,NaN
20,A.J. Burnett,2,2013,2014,NaN
1387,Aaron Harang,3,2013,2015,2015.0
21,Aaron Harang,1,2013,2013,NaN
278,Aaron Harang,2,2013,2014,NaN
1566,Adam Ottavino,9,2013,2021,2019.0
1697,Adam Ottavino,10,2013,2022,2019.0
22,Adam Ottavino,1,2013,2013,NaN


Para tener el mismo año de referencia, tendremos que usar los dataframes ordenados por jugador y luego el primer año como agentes libres

In [285]:
df_hitter_change = df_hitter_change.sort_values(by = ['Jugador', 'Primer_anio'])
df_fielder_change = df_fielder_change.sort_values(by = ['Jugador', 'Primer_anio'])

In [286]:
df_hitter_change['Primer_anio'] = df_hitter_change['Primer_anio'].ffill()
df_fielder_change['Primer_anio'] = df_fielder_change['Primer_anio'].ffill()

In [287]:
df_hitter_change[['Jugador','Aux_Anio','Anio_inicial','Anio_ref','Primer_anio']].head(20)

,Jugador,Aux_Anio,Anio_inicial,Anio_ref,Primer_anio
0,A.J. Burnett,3,2013,2015,2014.0
1,A.J. Burnett,4,2013,2016,2014.0
2,A.J. Burnett,2,2013,2014,2014.0
3,A.J. Burnett,1,2013,2013,2014.0
8,A.J. Ellis,7,2013,2019,2017.0
4,A.J. Ellis,6,2013,2018,2017.0
5,A.J. Ellis,3,2013,2015,2017.0
6,A.J. Ellis,2,2013,2014,2017.0
7,A.J. Ellis,1,2013,2013,2017.0
9,A.J. Ellis,5,2013,2017,2017.0


In [288]:
df_fielder_change[['Jugador','Aux_Anio','Anio_inicial','Anio_ref','Primer_anio']].head(20)

,Jugador,Aux_Anio,Anio_inicial,Anio_ref,Primer_anio
1343,A.J. Burnett,3,2013,2015,2014.0
1386,A.J. Burnett,4,2013,2016,2014.0
19,A.J. Burnett,1,2013,2013,2014.0
20,A.J. Burnett,2,2013,2014,2014.0
1387,Aaron Harang,3,2013,2015,2015.0
21,Aaron Harang,1,2013,2013,2015.0
278,Aaron Harang,2,2013,2014,2015.0
1566,Adam Ottavino,9,2013,2021,2019.0
1697,Adam Ottavino,10,2013,2022,2019.0
22,Adam Ottavino,1,2013,2013,2019.0


Ahora, creemos la columna que sirve como referencia para aplicar el _DID_ escalonado

In [289]:
df_hitter_change['Anio_did'] = df_hitter_change['Anio_ref'] - df_hitter_change['Primer_anio']
df_fielder_change['Anio_did'] = df_fielder_change['Anio_ref'] - df_fielder_change['Primer_anio']

In [290]:
df_hitter_change[['Anio_ref','Anio_did']].head(10)

,Anio_ref,Anio_did
0,2015,1.0
1,2016,2.0
2,2014,0.0
3,2013,-1.0
8,2019,2.0
4,2018,1.0
5,2015,-2.0
6,2014,-3.0
7,2013,-4.0
9,2017,0.0


In [291]:
df_fielder_change[['Anio_ref','Anio_did']].head(10)

,Anio_ref,Anio_did
1343,2015,1.0
1386,2016,2.0
19,2013,-1.0
20,2014,0.0
1387,2015,0.0
21,2013,-2.0
278,2014,-1.0
1566,2021,2.0
1697,2022,3.0
22,2013,-6.0


In [292]:
df_hitter_change.reset_index(drop = True,
                             inplace = True)
df_fielder_change.reset_index(drop = True,
                              inplace = True)

# Variables del Modelo Empírico

In [293]:
empiric_panel_hitter = df_hitter_change.copy()
empiric_panel_pitcher = df_fielder_change.copy()

Veamos algunas estadísticas e información que contienen las bases de datos

In [294]:
print(empiric_panel_hitter.shape)

(1852, 149)


In [295]:
print(empiric_panel_pitcher.shape)

(1789, 151)


Las posiciones que hay en cada base de datos

In [296]:
empiric_panel_hitter['Posicion_t'].unique()

array(['SP', 'C', 'CF', 'LF', 'RF', 'DH', '1B', '2B', 'SS', '3B', 'RP',
       'OF'], dtype=object)

In [297]:
empiric_panel_pitcher['Posicion_t'].unique()

array(['SP', 'RP', 'RP/CL', 'CL', 'SS', 'C'], dtype=object)

Ordenemos las bases de datos  por nombre y año

In [298]:
# Hitter
empiric_panel_hitter = empiric_panel_hitter.sort_values(by = ['Jugador','Anio_t'], ascending=True)
empiric_panel_hitter.reset_index(drop = True, inplace = True)

# Pitcher
empiric_panel_pitcher = empiric_panel_pitcher.sort_values(by = ['Jugador','Anio_t'], ascending=True)
empiric_panel_pitcher.reset_index(drop = True, inplace = True)

In [299]:
empiric_panel_hitter[['Jugador','Anio_t']].head()

,Jugador,Anio_t
0,A.J. Burnett,2013
1,A.J. Burnett,2013
2,A.J. Burnett,2014
3,A.J. Burnett,2015
4,A.J. Ellis,2013


In [300]:
empiric_panel_pitcher[['Jugador','Anio_t']].head()

,Jugador,Anio_t
0,A.J. Burnett,2013
1,A.J. Burnett,2013
2,A.J. Burnett,2014
3,A.J. Burnett,2015
4,Aaron Harang,2013


In [301]:
"""hitter_names = empiric_panel_hitter.columns
for index in range(0,len(hitter_names)):
    print("Name: " + str(hitter_names[index]))
    print("index: " + str(index))"""

'hitter_names = empiric_panel_hitter.columns\nfor index in range(0,len(hitter_names)):\n    print("Name: " + str(hitter_names[index]))\n    print("index: " + str(index))'

Obtengamos los índices de las columnas de interes

In [302]:
hitter_regular_stats = ['At_bats_2_t', 'At_bats_t',
                        'Bateos_2_t', 'Bateos_t',
                        'Bateos_promedio_2_t', 'Bateos_promedio_t',
                        'Dobles_2_t', 'Dobles_t',
                        'Home_runs_2_t', 'Home_runs_t',
                        'Juegos_iniciados_2_t', 'Juegos_iniciados_t', 
                        'Porcentaje_On_base_plus_slugging_2_t', 'Porcentaje_On_base_plus_slugging_t',
                        'Porcentaje_on_base_2_t', 'Porcentaje_on_base_t',
                        'Porcentaje_slugging_2_t', 'Porcentaje_slugging_t',
                        'Runs_batted_in_2_t', 'Runs_batted_in_t',
                        'Triples_2_t', 'Triples_t',
                        'WAR_2_t', 'WAR_t',
                        
                        'At_bats_2_t_1', 'At_bats_t_1',
                        'Bateos_2_t_1', 'Bateos_t_1',
                        'Bateos_promedio_2_t_1', 'Bateos_promedio_t_1',
                        'Dobles_2_t_1', 'Dobles_t_1',
                        'Home_runs_2_t_1', 'Home_runs_t_1',
                        'Juegos_iniciados_2_t_1', 'Juegos_iniciados_t_1', 
                        'Porcentaje_On_base_plus_slugging_2_t_1', 'Porcentaje_On_base_plus_slugging_t_1',
                        'Porcentaje_on_base_2_t_1', 'Porcentaje_on_base_t_1',
                        'Porcentaje_slugging_2_t_1', 'Porcentaje_slugging_t_1',
                        'Runs_batted_in_2_t_1', 'Runs_batted_in_t_1',
                        'Triples_2_t_1', 'Triples_t_1',
                        'WAR_2_t_1', 'WAR_t_1']
hitter_regular_stats = sorted(hitter_regular_stats)

In [303]:
# Hitter
for stat in range(0,len(hitter_regular_stats)):
    # Variables auxiliares
    stat_name = hitter_regular_stats[stat]
    max_stat_name = stat_name + '_H'
    min_stat_name = stat_name + '_L'
    
    # Máximos por equipo
    max_stat = pd.DataFrame({"Acronimo_t":empiric_panel_hitter.groupby(by = "Acronimo_t")[stat_name].max().index,
                             max_stat_name: empiric_panel_hitter.groupby(by = "Acronimo_t")[stat_name].max().values})
    # Mínimos por equipo
    min_stat = pd.DataFrame({"Acronimo_t":empiric_panel_hitter.groupby(by = "Acronimo_t")[stat_name].min().index,
                             min_stat_name: empiric_panel_hitter.groupby(by = "Acronimo_t")[stat_name].min().values})
    empiric_panel_hitter = empiric_panel_hitter.merge(max_stat, on = "Acronimo_t",
                                                      how = "left")
    empiric_panel_hitter = empiric_panel_hitter.merge(min_stat, on = "Acronimo_t",
                                                      how = "left")

In [304]:
"""hitter_names = empiric_panel_hitter.columns
for index in range(0,len(hitter_names)):
    print("Name: " + str(hitter_names[index]))
    print("index: " + str(index))"""

'hitter_names = empiric_panel_hitter.columns\nfor index in range(0,len(hitter_names)):\n    print("Name: " + str(hitter_names[index]))\n    print("index: " + str(index))'

In [305]:
start = empiric_panel_hitter.columns.get_loc('At_bats_2_t_H')
end = empiric_panel_hitter.columns.get_loc('WAR_t_1_L') + 1
empiric_panel_hitter.iloc[:,start:end]

,At_bats_2_t_H,At_bats_2_t_L,At_bats_2_t_1_H,At_bats_2_t_1_L,At_bats_t_H,At_bats_t_L,At_bats_t_1_H,At_bats_t_1_L,Bateos_2_t_H,Bateos_2_t_L,...,Triples_t_1_H,Triples_t_1_L,WAR_2_t_H,WAR_2_t_L,WAR_2_t_1_H,WAR_2_t_1_L,WAR_t_H,WAR_t_L,WAR_t_1_H,WAR_t_1_L
0,335241.0,1.0,335241.0,1.0,579.0,1.0,579.0,1.0,27556,1,...,10.0,0.0,31.5844,0.0121,31.5844,0.0016,5.62,0.11,5.62,0.04
1,335241.0,1.0,335241.0,1.0,579.0,1.0,579.0,1.0,27556,1,...,10.0,0.0,31.5844,0.0121,31.5844,0.0016,5.62,0.11,5.62,0.04
2,361201.0,1.0,344569.0,1.0,601.0,1.0,587.0,1.0,33856,0,...,6.0,0.0,38.8129,0.0009,20.5209,0.0009,6.23,0.03,4.53,0.03
3,335241.0,1.0,335241.0,1.0,579.0,1.0,579.0,1.0,27556,1,...,10.0,0.0,31.5844,0.0121,31.5844,0.0016,5.62,0.11,5.62,0.04
4,399424.0,1.0,396900.0,9.0,632.0,1.0,630.0,3.0,39601,0,...,8.0,0.0,78.8544,0.0016,37.2100,0.0001,8.88,0.04,6.10,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,399424.0,1.0,396900.0,9.0,632.0,1.0,630.0,3.0,39601,0,...,8.0,0.0,78.8544,0.0016,37.2100,0.0001,8.88,0.04,6.10,0.01
1848,370881.0,1.0,384400.0,4.0,609.0,1.0,620.0,2.0,36864,0,...,9.0,0.0,47.1969,0.0001,78.8544,0.0144,6.87,0.01,8.88,0.12
1849,370881.0,1.0,384400.0,4.0,609.0,1.0,620.0,2.0,36864,0,...,9.0,0.0,47.1969,0.0001,78.8544,0.0144,6.87,0.01,8.88,0.12
1850,370881.0,1.0,384400.0,4.0,609.0,1.0,620.0,2.0,36864,0,...,9.0,0.0,47.1969,0.0001,78.8544,0.0144,6.87,0.01,8.88,0.12


Repitamos el mismo proceso para los lanzadores

In [306]:
"""pitcher_names = empiric_panel_pitcher.columns
for index in range(0,len(pitcher_names)):
    print("Name: " + str(pitcher_names[index]))
    print("index: " + str(index))"""

'pitcher_names = empiric_panel_pitcher.columns\nfor index in range(0,len(pitcher_names)):\n    print("Name: " + str(pitcher_names[index]))\n    print("index: " + str(index))'

Repitamos el mismo proceso para los filderos

In [307]:
pitcher_regular_stats = ['Bateos_2_t', 'Bateos_t',
                        'Carreras_2_t', 'Carreras_t',
                        'Carreras_ganadas_2_t', 'Carreras_ganadas_t',
                        'Comando_2_t', 'Comando_t',
                        'Control_2_t', 'Control_t',
                        'Dominio_2_t', 'Dominio_t', 
                        'ERA_2_t', 'ERA_t',
                        'Inning_pitched_2_t', 'Inning_pitched_t',
                        'Losses_2_t', 'Losses_t',
                        'Saves_2_t', 'Saves_t',
                        'Strike_outs_2_t', 'Strike_outs_t',
                        'WAR_2_t', 'WAR_t',
                        'WHIP_2_t', 'WHIP_t',
                        'Walks_2_t', 'Walks_t',
                        'Wins_2_t', 'Wins_t',
    
                        'Bateos_2_t_1', 'Bateos_t_1',
                        'Carreras_2_t_1', 'Carreras_t_1',
                        'Carreras_ganadas_2_t_1', 'Carreras_ganadas_t_1',
                        'Comando_2_t_1', 'Comando_t_1',
                        'Control_2_t_1', 'Control_t_1',
                        'Dominio_2_t_1', 'Dominio_t_1', 
                        'ERA_2_t_1', 'ERA_t_1',
                        'Inning_pitched_2_t_1', 'Inning_pitched_t_1',
                        'Losses_2_t_1', 'Losses_t_1',
                        'Saves_2_t_1', 'Saves_t_1',
                        'Strike_outs_2_t_1', 'Strike_outs_t_1',
                        'WAR_2_t_1', 'WAR_t_1',
                        'WHIP_2_t_1', 'WHIP_t_1',
                        'Walks_2_t_1', 'Walks_t_1',
                        'Wins_2_t_1', 'Wins_t_1']
pitcher_regular_stats = sorted(pitcher_regular_stats)

In [308]:
# Pitcheer
for stat in range(0,len(pitcher_regular_stats)):
    # Variables auxiliares
    stat_name = pitcher_regular_stats[stat]
    max_stat_name = stat_name + '_H'
    min_stat_name = stat_name + '_L'
    
    # Máximos por equipo
    max_stat = pd.DataFrame({"Acronimo_t":empiric_panel_pitcher.groupby(by = "Acronimo_t")[stat_name].max().index,
                             max_stat_name: empiric_panel_pitcher.groupby(by = "Acronimo_t")[stat_name].max().values})
    # Mínimos por equipo
    min_stat = pd.DataFrame({"Acronimo_t":empiric_panel_pitcher.groupby(by = "Acronimo_t")[stat_name].min().index,
                             min_stat_name: empiric_panel_pitcher.groupby(by = "Acronimo_t")[stat_name].min().values})
    empiric_panel_pitcher = empiric_panel_pitcher.merge(max_stat, on = "Acronimo_t",
                                                        how = "left")
    empiric_panel_pitcher = empiric_panel_pitcher.merge(min_stat, on = "Acronimo_t",
                                                        how = "left")

In [309]:
"""pitcher_names = empiric_panel_pitcher.columns
for index in range(0,len(pitcher_names)):
    print("Name: " + str(pitcher_names[index]))
    print("index: " + str(index))"""

'pitcher_names = empiric_panel_pitcher.columns\nfor index in range(0,len(pitcher_names)):\n    print("Name: " + str(pitcher_names[index]))\n    print("index: " + str(index))'

In [310]:
start = empiric_panel_pitcher.columns.get_loc('Bateos_2_t_H')
end = empiric_panel_pitcher.columns.get_loc('Wins_t_1_L') + 1
empiric_panel_pitcher.iloc[:,start:end]

,Bateos_2_t_H,Bateos_2_t_L,Bateos_2_t_1_H,Bateos_2_t_1_L,Bateos_t_H,Bateos_t_L,Bateos_t_1_H,Bateos_t_1_L,Carreras_2_t_H,Carreras_2_t_L,...,Walks_t_1_H,Walks_t_1_L,Wins_2_t_H,Wins_2_t_L,Wins_2_t_1_H,Wins_2_t_1_L,Wins_t_H,Wins_t_L,Wins_t_1_H,Wins_t_1_L
0,41209,9,42025.0,100.0,203,3,205.0,10.0,9604,1,...,96.0,3.0,361,0,361.0,0.0,19,0,19.0,0.0
1,41209,9,42025.0,100.0,203,3,205.0,10.0,9604,1,...,96.0,3.0,361,0,361.0,0.0,19,0,19.0,0.0
2,45796,4,46225.0,4.0,214,2,215.0,2.0,14884,0,...,71.0,0.0,144,0,289.0,0.0,12,0,17.0,0.0
3,41209,9,42025.0,100.0,203,3,205.0,10.0,9604,1,...,96.0,3.0,361,0,361.0,0.0,19,0,19.0,0.0
4,38416,0,39204.0,0.0,196,0,198.0,0.0,10201,0,...,85.0,0.0,196,0,324.0,0.0,14,0,18.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784,55696,1,55696.0,16.0,236,1,236.0,4.0,15376,0,...,74.0,0.0,324,0,225.0,0.0,18,0,15.0,0.0
1785,38416,0,39204.0,0.0,196,0,198.0,0.0,10201,0,...,85.0,0.0,196,0,324.0,0.0,14,0,18.0,0.0
1786,38416,0,39204.0,0.0,196,0,198.0,0.0,10201,0,...,85.0,0.0,196,0,324.0,0.0,14,0,18.0,0.0
1787,38416,0,39204.0,0.0,196,0,198.0,0.0,10201,0,...,85.0,0.0,196,0,324.0,0.0,14,0,18.0,0.0


In [311]:
empiric_panel_hitter.shape

(1852, 245)

In [312]:
empiric_panel_pitcher.shape

(1789, 271)

### Filtrando jugadores con más de dos años de observaciones en nuestro panel

Para ser consistentes, las columnas que contienen datos de tipo *string* las imputaremos con la palabra *No* ya que representará que no tenía equipo, ni posición, etc.

In [313]:
empiric_panel_hitter.select_dtypes(include =['object'],
                                   exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1852 entries, 0 to 1851
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Acronimo_t       1852 non-null   object
 1   Acronimo_t_1     1844 non-null   object
 2   Agente_libre     1852 non-null   object
 3   Equipo_anterior  1852 non-null   object
 4   Equipo_t         1852 non-null   object
 5   Equipo_t_1       1844 non-null   object
 6   Estado_t         1852 non-null   object
 7   Jugador          1852 non-null   object
 8   Posicion_t       1852 non-null   object
 9   Posicion_t_1     1844 non-null   object
 10  Tratamiento      1852 non-null   object
dtypes: object(11)
memory usage: 173.6+ KB


In [314]:
empiric_panel_hitter[['Acronimo_t',
                      'Equipo_anterior',
                      'Equipo_t',
                      'Estado_t',
                      'Posicion_t',
                      'Acronimo_t_1',
                      'Equipo_t_1',
                      'Posicion_t_1']] = \
empiric_panel_hitter[['Acronimo_t',
                      'Equipo_anterior',
                      'Equipo_t',
                      'Estado_t',
                      'Posicion_t',
                      'Acronimo_t_1',
                      'Equipo_t_1',
                      'Posicion_t_1']].fillna('No')

Veamos si funcionó la imputación

In [315]:
empiric_panel_hitter.select_dtypes(include =['object'],
                                   exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1852 entries, 0 to 1851
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Acronimo_t       1852 non-null   object
 1   Acronimo_t_1     1852 non-null   object
 2   Agente_libre     1852 non-null   object
 3   Equipo_anterior  1852 non-null   object
 4   Equipo_t         1852 non-null   object
 5   Equipo_t_1       1852 non-null   object
 6   Estado_t         1852 non-null   object
 7   Jugador          1852 non-null   object
 8   Posicion_t       1852 non-null   object
 9   Posicion_t_1     1852 non-null   object
 10  Tratamiento      1852 non-null   object
dtypes: object(11)
memory usage: 173.6+ KB


Ahora con los lanzadores

In [316]:
empiric_panel_pitcher.select_dtypes(include =['object'],
                                    exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1789 entries, 0 to 1788
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Acronimo_t       1789 non-null   object
 1   Acronimo_t_1     1782 non-null   object
 2   Agente_libre     1789 non-null   object
 3   Equipo_anterior  1789 non-null   object
 4   Equipo_t         1789 non-null   object
 5   Equipo_t_1       1782 non-null   object
 6   Estado_t         1789 non-null   object
 7   Jugador          1789 non-null   object
 8   Posicion_t       1789 non-null   object
 9   Posicion_t_1     1782 non-null   object
 10  Tratamiento      1789 non-null   object
dtypes: object(11)
memory usage: 167.7+ KB


In [317]:
empiric_panel_pitcher[['Acronimo_t',
                       'Equipo_anterior',
                       'Equipo_t',
                       'Estado_t',
                       'Posicion_t',
                       'Acronimo_t_1',
                       'Equipo_t_1',
                       'Posicion_t_1']] = \
empiric_panel_pitcher[['Acronimo_t',
                       'Equipo_anterior',
                       'Equipo_t',
                       'Estado_t',
                       'Posicion_t',
                       'Acronimo_t_1',
                       'Equipo_t_1',
                       'Posicion_t_1']].fillna('No')

In [318]:
empiric_panel_pitcher.select_dtypes(include =['object'],
                                    exclude = ['int64','float64']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1789 entries, 0 to 1788
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Acronimo_t       1789 non-null   object
 1   Acronimo_t_1     1789 non-null   object
 2   Agente_libre     1789 non-null   object
 3   Equipo_anterior  1789 non-null   object
 4   Equipo_t         1789 non-null   object
 5   Equipo_t_1       1789 non-null   object
 6   Estado_t         1789 non-null   object
 7   Jugador          1789 non-null   object
 8   Posicion_t       1789 non-null   object
 9   Posicion_t_1     1789 non-null   object
 10  Tratamiento      1789 non-null   object
dtypes: object(11)
memory usage: 167.7+ KB


En ambos casos, la imputación fue un éxito. No importa que en periodos de contratación de haya puesto *No* ya que con esto evitamos problemas con posibles instrumentos a partir de dummies.

Por otro lado, repitamos lo mismo para las columnas numéricas, imputaremos 0 ya que refleja la ausencia de desempeño. Sin embargo, la columna que contiene la estadística más alta de cada medida de desempeño se imputará por separado debido a que causará problemas cuando se imponga la trnasformación para el modelo.

In [319]:
# Add string to every element in a list
# Hitter
hitter_high_stat = [stat + '_H' for stat in hitter_regular_stats]
hitter_low_stat = [stat + '_L' for stat in hitter_regular_stats]
# Fielder
pitcher_high_stat = [stat + '_H' for stat in pitcher_regular_stats]
pitcher_low_stat = [stat + '_L' for stat in pitcher_regular_stats]

In [320]:
empiric_panel_hitter[hitter_high_stat] = empiric_panel_hitter[hitter_high_stat].fillna(method = 'ffill')
empiric_panel_pitcher[pitcher_high_stat] = empiric_panel_pitcher[pitcher_high_stat].fillna(method = 'ffill')

In [321]:
empiric_panel_hitter[hitter_high_stat].head(20)

,At_bats_2_t_H,At_bats_2_t_1_H,At_bats_t_H,At_bats_t_1_H,Bateos_2_t_H,Bateos_2_t_1_H,Bateos_promedio_2_t_H,Bateos_promedio_2_t_1_H,Bateos_promedio_t_H,Bateos_promedio_t_1_H,...,Runs_batted_in_t_H,Runs_batted_in_t_1_H,Triples_2_t_H,Triples_2_t_1_H,Triples_t_H,Triples_t_1_H,WAR_2_t_H,WAR_2_t_1_H,WAR_t_H,WAR_t_1_H
0,335241.0,335241.0,579.0,579.0,27556,27556.0,1.000000,0.099225,1.000,0.315,...,100.0,100.0,100,100.0,10,10.0,31.5844,31.5844,5.62,5.62
1,335241.0,335241.0,579.0,579.0,27556,27556.0,1.000000,0.099225,1.000,0.315,...,100.0,100.0,100,100.0,10,10.0,31.5844,31.5844,5.62,5.62
2,361201.0,344569.0,601.0,587.0,33856,26896.0,0.093636,1.000000,0.306,1.000,...,94.0,100.0,49,36.0,7,6.0,38.8129,20.5209,6.23,4.53
3,335241.0,335241.0,579.0,579.0,27556,27556.0,1.000000,0.099225,1.000,0.315,...,100.0,100.0,100,100.0,10,10.0,31.5844,31.5844,5.62,5.62
4,399424.0,396900.0,632.0,630.0,39601,37249.0,0.119025,0.119025,0.345,0.345,...,107.0,95.0,64,64.0,8,8.0,78.8544,37.2100,8.88,6.10
5,399424.0,396900.0,632.0,630.0,39601,37249.0,0.119025,0.119025,0.345,0.345,...,107.0,95.0,64,64.0,8,8.0,78.8544,37.2100,8.88,6.10
6,399424.0,396900.0,632.0,630.0,39601,37249.0,0.119025,0.119025,0.345,0.345,...,107.0,95.0,64,64.0,8,8.0,78.8544,37.2100,8.88,6.10
7,399424.0,396900.0,632.0,630.0,39601,37249.0,0.119025,0.119025,0.345,0.345,...,107.0,95.0,64,64.0,8,8.0,78.8544,37.2100,8.88,6.10
8,361201.0,344569.0,601.0,587.0,33856,26896.0,0.093636,1.000000,0.306,1.000,...,94.0,100.0,49,36.0,7,6.0,38.8129,20.5209,6.23,4.53
9,404496.0,379456.0,636.0,616.0,35721,31329.0,0.094864,0.092416,0.308,0.304,...,124.0,86.0,100,100.0,10,10.0,31.6969,19.0096,5.63,4.36


In [322]:
empiric_panel_hitter.fillna(0, inplace = True)
empiric_panel_pitcher.fillna(0, inplace = True)

Verifiquemos si queda alguna columna con alguna entrada tipo *NaN*:

In [323]:
# Hitter
hitter_nan = empiric_panel_hitter.isna().any()
hitter_name = empiric_panel_hitter.columns
for con in range(0, len(hitter_nan)):
    if hitter_nan[con]:
        print("Name: " + str(hitter_name[con]))

In [324]:
# Pitcher
pitcher_nan = empiric_panel_pitcher.isna().any()
pitcher_name = empiric_panel_pitcher.columns
for con in range(0, len(pitcher_nan)):
    if pitcher_nan[con]:
        print("Name: " + str(pitcher_name[con]))

Obtengamos el máximo de cada una de las medidas de desempeño, de periodos *t_1*, por equipo que han obtenido a lo largo de todas la temporadas

Para evitar problemas con el tipo de *ID*, creemos una que sea numérica para evitar usar los nombres de los jugadores

In [325]:
# Hitter
empiric_panel_hitter['id'] =  empiric_panel_hitter.groupby(['Jugador']).ngroup()
empiric_panel_hitter.reset_index(drop = True, inplace = True)
# Pitcher
empiric_panel_pitcher['id'] =  empiric_panel_pitcher.groupby(['Jugador']).ngroup()
empiric_panel_pitcher.reset_index(drop = True, inplace = True)

Obtengamos la transformación para obtener la $Y$ del modelo empírico a partir de los rezagos de las raices de dichos salarios

In [326]:
# Función de rezagos de raices
def sqrt_dif(X):
    S = []
    for i in range(0, len(X)-1):
        d = np.sqrt(X[i+1])-np.sqrt(X[i])
        S.append(d)
    try:
        S.append(d)
    except: 
        S.append(0)
    return S

Haremos un ciclo para hacer sobre todas las posibilidades de variables de compensación salarial

In [327]:
compensations = ['Sueldo_ajustado_norm_t', 'Sueldo_regular_norm_t',
                 'Sueldo_base_norm_t', 'TVS_t',
                 'ln_Sueldo_ajustado_t', 'ln_Sueldo_regular_t']

In [328]:
for i in range(len(compensations)):
    Y_hitter = []
    for p in empiric_panel_hitter["id"].unique():
        # Filtremos todos los sueldos (ln) de cada jugador por separado
        X = empiric_panel_hitter[empiric_panel_hitter["id"] == p][compensations[i]].values
        # Aplicación de la función
        S = sqrt_dif(X)
        # Añadimos los datos de manera ordenada
        Y_hitter = np.concatenate((Y_hitter, S))
    # Agregamos la columna:
    Y_name = 'Y_' + compensations[i]
    empiric_panel_hitter[Y_name] = Y_hitter

In [329]:
for i in range(len(compensations)):
    Y_pitcher = []
    for p in empiric_panel_pitcher["id"].unique():
        # Filtremos todos los sueldos (ln) de cada jugador por separado
        X = empiric_panel_pitcher[empiric_panel_pitcher["id"] == p][compensations[i]].values
        # Aplicación de la función
        S = sqrt_dif(X)
        # Añadimos los datos de manera ordenada
        Y_pitcher = np.concatenate((Y_pitcher, S))
    # Agregamos la columna:
    Y_name = 'Y_' + compensations[i]
    empiric_panel_pitcher[Y_name] = Y_pitcher

Contruyamos las dummy *I* del modelo empírico

In [330]:
# Long of the stats
end_hitter_name = int((len(hitter_high_stat) + len(hitter_low_stat))/2)

In [331]:
hitter_regular_stats

['At_bats_2_t',
 'At_bats_2_t_1',
 'At_bats_t',
 'At_bats_t_1',
 'Bateos_2_t',
 'Bateos_2_t_1',
 'Bateos_promedio_2_t',
 'Bateos_promedio_2_t_1',
 'Bateos_promedio_t',
 'Bateos_promedio_t_1',
 'Bateos_t',
 'Bateos_t_1',
 'Dobles_2_t',
 'Dobles_2_t_1',
 'Dobles_t',
 'Dobles_t_1',
 'Home_runs_2_t',
 'Home_runs_2_t_1',
 'Home_runs_t',
 'Home_runs_t_1',
 'Juegos_iniciados_2_t',
 'Juegos_iniciados_2_t_1',
 'Juegos_iniciados_t',
 'Juegos_iniciados_t_1',
 'Porcentaje_On_base_plus_slugging_2_t',
 'Porcentaje_On_base_plus_slugging_2_t_1',
 'Porcentaje_On_base_plus_slugging_t',
 'Porcentaje_On_base_plus_slugging_t_1',
 'Porcentaje_on_base_2_t',
 'Porcentaje_on_base_2_t_1',
 'Porcentaje_on_base_t',
 'Porcentaje_on_base_t_1',
 'Porcentaje_slugging_2_t',
 'Porcentaje_slugging_2_t_1',
 'Porcentaje_slugging_t',
 'Porcentaje_slugging_t_1',
 'Runs_batted_in_2_t',
 'Runs_batted_in_2_t_1',
 'Runs_batted_in_t',
 'Runs_batted_in_t_1',
 'Triples_2_t',
 'Triples_2_t_1',
 'Triples_t',
 'Triples_t_1',
 'WAR_2_

In [332]:
# Hitter
for sport_stat in range(0,end_hitter_name):
    I_hitter = []
    for y,max_stat,min_stat in zip(empiric_panel_hitter[hitter_regular_stats[sport_stat]],
                                   empiric_panel_hitter[hitter_high_stat[sport_stat]],
                                   empiric_panel_hitter[hitter_low_stat[sport_stat]]):
        # Dummy condition
        if y > (max_stat + min_stat)/2:
            I_hitter.append(0)
        else: 
            I_hitter.append(1)
    
    I_name = hitter_regular_stats[sport_stat] + '_I'
    empiric_panel_hitter[I_name] = I_hitter

Veamos los resultados

In [333]:
"""hitter_names = empiric_panel_hitter.columns
for index in range(0,len(hitter_names)):
    print("Name: " + str(hitter_names[index]))
    print("index: " + str(index))"""

'hitter_names = empiric_panel_hitter.columns\nfor index in range(0,len(hitter_names)):\n    print("Name: " + str(hitter_names[index]))\n    print("index: " + str(index))'

In [334]:
# Hitter
hitter_nan = empiric_panel_hitter.isna().any()
hitter_name = empiric_panel_hitter.columns
for con in range(0, len(hitter_nan)):
    if hitter_nan[con]:
        print("Name: " + str(hitter_name[con]))

In [335]:
start = empiric_panel_hitter.columns.get_loc('At_bats_2_t_I')
end = empiric_panel_hitter.columns.get_loc('WAR_t_1_I') + 1
empiric_panel_hitter.iloc[:,start:end]

,At_bats_2_t_I,At_bats_2_t_1_I,At_bats_t_I,At_bats_t_1_I,Bateos_2_t_I,Bateos_2_t_1_I,Bateos_promedio_2_t_I,Bateos_promedio_2_t_1_I,Bateos_promedio_t_I,Bateos_promedio_t_1_I,...,Runs_batted_in_t_I,Runs_batted_in_t_1_I,Triples_2_t_I,Triples_2_t_1_I,Triples_t_I,Triples_t_1_I,WAR_2_t_I,WAR_2_t_1_I,WAR_t_I,WAR_t_1_I
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,0,0,1,1,1,0,0,0,...,1,0,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,1,1,1,1,1,1,1,1,0,0,...,1,1,1,1,1,1,0,0,0,0
1848,1,1,1,1,1,1,1,1,0,0,...,1,1,1,1,1,1,1,0,1,0
1849,1,1,1,1,1,1,1,1,0,0,...,1,1,1,1,1,1,0,1,0,1
1850,1,1,1,1,1,1,0,1,0,0,...,1,1,1,1,1,1,1,1,0,0


Repitamos el mismo proceso para los lanzadores

In [336]:
# Long of stats
end_pitcher_name = int((len(pitcher_high_stat) + len(pitcher_low_stat))/2)

In [337]:
# Pitcher
for sport_stat in range(0,end_pitcher_name):
    I_pitcher = []
    for y,max_stat,min_stat in zip(empiric_panel_pitcher[pitcher_regular_stats[sport_stat]],
                                   empiric_panel_pitcher[pitcher_high_stat[sport_stat]],
                                   empiric_panel_pitcher[pitcher_low_stat[sport_stat]]):
        if y > (max_stat + min_stat)/2:
            I_pitcher.append(0)
        else: 
            I_pitcher.append(1)
    
    I_name = pitcher_regular_stats[sport_stat] + '_I'
    empiric_panel_pitcher[I_name] = I_pitcher

In [338]:
"""pitcher_names = empiric_panel_pitcher.columns
for index in range(0,len(pitcher_names)):
    print("Name: " + str(pitcher_names[index]))
    print("index: " + str(index))"""

'pitcher_names = empiric_panel_pitcher.columns\nfor index in range(0,len(pitcher_names)):\n    print("Name: " + str(pitcher_names[index]))\n    print("index: " + str(index))'

In [339]:
# Pitcher
pitcher_nan = empiric_panel_pitcher.isna().any()
pitcher_name = empiric_panel_pitcher.columns
for con in range(0, len(pitcher_nan)):
    if pitcher_nan[con]:
        print("Name: " + str(pitcher_name[con]))

In [340]:
start = empiric_panel_pitcher.columns.get_loc('Bateos_2_t_I')
end = empiric_panel_pitcher.columns.get_loc('Wins_t_1_I') + 1
empiric_panel_pitcher.iloc[:,start:end]

,Bateos_2_t_I,Bateos_2_t_1_I,Bateos_t_I,Bateos_t_1_I,Carreras_2_t_I,Carreras_2_t_1_I,Carreras_ganadas_2_t_I,Carreras_ganadas_2_t_1_I,Carreras_ganadas_t_I,Carreras_ganadas_t_1_I,...,WHIP_t_I,WHIP_t_1_I,Walks_2_t_I,Walks_2_t_1_I,Walks_t_I,Walks_t_1_I,Wins_2_t_I,Wins_2_t_1_I,Wins_t_I,Wins_t_1_I
0,0,0,0,0,0,1,0,0,0,0,...,1,1,0,1,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,1,1,0,1,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,0
3,0,0,0,0,1,0,1,0,0,0,...,1,1,1,0,0,0,1,1,1,1
4,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784,1,1,0,0,1,0,1,0,1,0,...,1,1,1,1,1,1,1,0,1,0
1785,0,1,0,1,0,1,0,1,0,1,...,0,0,0,1,0,0,0,1,0,1
1786,0,1,0,1,1,1,0,1,0,1,...,0,0,1,1,0,1,0,1,0,1
1787,0,0,0,0,0,1,0,1,0,0,...,0,0,1,1,0,0,0,1,0,0


Obtengamos las variables auxiliares

In [341]:
# Good practice
hitter_high_stat = sorted(hitter_high_stat)
# Dummy
I_hitter = sorted([stat + '_I' for stat in hitter_regular_stats])

Verifiquemos que tengan la misma longitud

In [342]:
len(I_hitter) == len(hitter_high_stat) == len(hitter_regular_stats)

True

In [343]:
# Pitcher
for stat in range(0,len(hitter_regular_stats)):
    # Variable auxiliar
    X_hitter = []
    
    # Variables 
    i = (-1)**(empiric_panel_hitter[I_hitter[stat]])
    x = empiric_panel_hitter[hitter_regular_stats[stat]]/np.sqrt(empiric_panel_hitter[hitter_high_stat[stat]])
    X_hitter = i*x
    
    # X name
    name = 'X_' + hitter_regular_stats[stat]
    empiric_panel_hitter[name] = X_hitter

In [344]:
# Pitcher
hitter_nan = empiric_panel_hitter.isna().any()
hitter_name = empiric_panel_hitter.columns
for con in range(0, len(hitter_nan)):
    if hitter_nan[con]:
        print("Name: " + str(hitter_name[con]))

In [345]:
# Drop columns 'B' and 'C'
empiric_panel_hitter = empiric_panel_hitter.drop(['X_Triples_2_t',
                                                  'X_Triples_2_t_1',
                                                  'X_Triples_t',
                                                  'X_Triples_t_1'], axis=1)

In [346]:
# calculate the mean of each column
hitter_means = empiric_panel_hitter.mean()

# impute missing data with the mean value of the column
empiric_panel_hitter = empiric_panel_hitter.fillna(hitter_means)

In [347]:
# define a list of suffixes to drop
drop_suffix = ['_I', '_L', '_H']

# Use a list comprehension to filter the columns to drop
hitter_cols_to_drop = [col for col in empiric_panel_hitter.columns if col.endswith(tuple(drop_suffix))]

# Drop the selected columns
empiric_panel_hitter = empiric_panel_hitter.drop(hitter_cols_to_drop,
                                                 axis=1)

In [348]:
empiric_panel_hitter.shape

(1852, 200)

In [349]:
empiric_panel_hitter['Posicion_t'].value_counts()

SP    345
C     251
LF    183
2B    167
RF    150
CF    143
SS    138
1B    135
3B    126
DH    106
RP     95
OF     13
Name: Posicion_t, dtype: int64

Veamos todas las variables del panel

In [350]:
panel_columns = empiric_panel_hitter.columns

for name in panel_columns:
    print(name)

Acronimo_t
Acronimo_t_1
Agente_libre
Altura_t
Altura_t_1
Anio_de_agente_libre_t
Anio_de_agente_libre_t_1
Anio_inicial
Anio_ref
Anio_t
Anios_de_contrato_t
Anios_de_contrato_t_1
Antiguedad_t
Antiguedad_t_1
At-bats_2_t
At-bats_t
At_bats_2_t
At_bats_2_t_1
At_bats_t
At_bats_t_1
Aux_Anio
Bateos_2_t
Bateos_2_t_1
Bateos_promedio_2_t
Bateos_promedio_2_t_1
Bateos_promedio_t
Bateos_promedio_t_1
Bateos_t
Bateos_t_1
Bono_por_firma_t
Bono_por_firma_t_1
Cantidad de equipos_t
Cantidad_agentes_libres_t
Cantidad_agentes_libres_t_1
Cantidad_de_equipos_t
Cantidad_de_equipos_t_1
Dobles_2_t
Dobles_2_t_1
Dobles_t
Dobles_t_1
Edad_al_firmar_t
Edad_al_firmar_t_1
Edad_t
Equipo_anterior
Equipo_t
Equipo_t_1
Estado_t
Ganancias_t
Ganancias_t_1
Home-runs_2_t
Home-runs_t
Home_runs_2_t
Home_runs_2_t_1
Home_runs_t
Home_runs_t_1
Juegos totales_t
Juegos_iniciados_2_t
Juegos_iniciados_2_t_1
Juegos_iniciados_t
Juegos_iniciados_t_1
Juegos_t
Juegos_t_1
Juegos_totales_t
Juegos_totales_t_1
Jugador
Pago_efectivo_t
Pago_efectivo_

In [351]:
general_dynamic_path = 'ETL_Data/Panel/Cumulative/Bargaining_change/'

In [352]:
empiric_panel_hitter.to_csv(general_dynamic_path + 'panel_hitters_cum_did_all' + '.csv',
                            index = False)

In [353]:
values_to_drop = ['Triples_2_t',
                  'Triples_2_t_1',
                  'Triples_t',
                  'Triples_t_1']

for value in values_to_drop:
    hitter_regular_stats.remove(value)

Ahora, filtremos las columnas que contienen las medidas de desempeño de los periodos **t_1**. Para ello, encontremos primero los índices de dichas variables

In [354]:
Y_list = ['Y_Sueldo_ajustado_norm_t',
          'Y_Sueldo_regular_norm_t',
          'Y_Sueldo_base_norm_t',
          'Y_TVS_t',
          'Y_ln_Sueldo_ajustado_t',
          'Y_ln_Sueldo_regular_t']

In [355]:
# Pitcher
for y_name in Y_list:
    for stat in range(0,len(hitter_regular_stats)):
        # Stat
        print(colored(hitter_regular_stats[stat], "cyan"))

        # X name
        name = 'X_' + hitter_regular_stats[stat]

        # OLS variables
        Y = empiric_panel_hitter[y_name].tolist()
        X = empiric_panel_hitter[name].tolist()
        X = sm.add_constant(X)

        # Modelo
        model = sm.OLS(Y, X,
                       missing = 'drop').fit()
        print(model.summary())
        print("\n")

At_bats_2_t
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1706
Date:                Sat, 08 Apr 2023   Prob (F-statistic):              0.680
Time:                        14:37:13   Log-Likelihood:                 793.76
No. Observations:                1852   AIC:                            -1584.
Df Residuals:                    1850   BIC:                            -1572.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0035      0.004      0.

Repitamos el mismo procedimiento para los filderos

In [356]:
# Good practice
pitcher_high_stat = sorted(pitcher_high_stat)
# Dummy
I_pitcher = sorted([stat + '_I' for stat in pitcher_regular_stats])

Verifiquemos que tengan la misma longitud

In [357]:
len(I_pitcher) == len(pitcher_high_stat) == len(pitcher_regular_stats)

True

In [358]:
# Pitcher
for stat in range(0,len(pitcher_regular_stats)):
    # Variable auxiliar
    X_pitcher = []
    
    # Variables 
    i = (-1)**(empiric_panel_pitcher[I_pitcher[stat]])
    x = empiric_panel_pitcher[pitcher_regular_stats[stat]]/np.sqrt(empiric_panel_pitcher[pitcher_high_stat[stat]])
    X_pitcher = i*x
    
    # X name
    name = 'X_' + pitcher_regular_stats[stat]
    empiric_panel_pitcher[name] = X_pitcher

In [359]:
# Pitcher
pitcher_nan = empiric_panel_pitcher.isna().any()
pitcher_name = empiric_panel_pitcher.columns
for con in range(0, len(pitcher_nan)):
    if pitcher_nan[con]:
        print("Name: " + str(pitcher_name[con]))

In [360]:
# Drop columns 'B' and 'C'
empiric_panel_pitcher = empiric_panel_pitcher.drop(['X_Saves_2_t',
                                                    'X_Saves_2_t_1',
                                                    'X_Saves_t',
                                                    'X_Saves_t_1'], axis=1)

In [361]:
values_to_drop = ['Saves_2_t',
                  'Saves_2_t_1',
                  'Saves_t',
                  'Saves_t_1']

for value in values_to_drop:
    pitcher_regular_stats.remove(value)

Imputemos dichas columnas con la media

In [362]:
# calculate the mean of each column
fielder_means = empiric_panel_pitcher.mean()

# impute missing data with the mean value of the column
empiric_panel_pitcher = empiric_panel_pitcher.fillna(fielder_means)

Tenemos que borrar las columnas que terminan con '_L', '_H' o '_I'

In [363]:
# define a list of suffixes to drop
drop_suffix = ['_I', '_L', '_H']

# Use a list comprehension to filter the columns to drop
fielder_cols_to_drop = [col for col in empiric_panel_pitcher.columns if col.endswith(tuple(drop_suffix))]

# Drop the selected columns
empiric_panel_pitcher = empiric_panel_pitcher.drop(fielder_cols_to_drop,
                                                   axis = 1)

In [364]:
empiric_panel_pitcher.shape

(1789, 214)

In [365]:
empiric_panel_pitcher['Posicion_t'].value_counts()

RP       907
SP       785
RP/CL     78
SS        17
CL         1
C          1
Name: Posicion_t, dtype: int64

In [366]:
panel_columns = empiric_panel_pitcher.columns

for name in panel_columns:
    print(name)

Acronimo_t
Acronimo_t_1
Agente_libre
Altura_t
Altura_t_1
Anio_de_agente_libre_t
Anio_de_agente_libre_t_1
Anio_inicial
Anio_ref
Anio_t
Anios_de_contrato_t
Anios_de_contrato_t_1
Antiguedad_t
Antiguedad_t_1
Aux_Anio
Bateos_2_t
Bateos_2_t_1
Bateos_t
Bateos_t_1
Bono_por_firma_t
Bono_por_firma_t_1
Cantidad de equipos_t
Cantidad_agentes_libres_t
Cantidad_agentes_libres_t_1
Cantidad_de_equipos_t
Cantidad_de_equipos_t_1
Carreras_2_t
Carreras_2_t_1
Carreras_ganadas_2_t
Carreras_ganadas_2_t_1
Carreras_ganadas_t
Carreras_ganadas_t_1
Carreras_t
Carreras_t_1
Comando_2_t
Comando_2_t_1
Comando_t
Comando_t_1
Control_2_t
Control_2_t_1
Control_t
Control_t_1
Dominio_2_t
Dominio_2_t_1
Dominio_t
Dominio_t_1
ERA_2_t
ERA_2_t_1
ERA_t
ERA_t_1
Edad_al_firmar_t
Edad_al_firmar_t_1
Edad_t
Equipo_anterior
Equipo_t
Equipo_t_1
Estado_t
Ganancias_t
Ganancias_t_1
Inning_pitched_2_t
Inning_pitched_2_t_1
Inning_pitched_t
Inning_pitched_t_1
Juegos totales_t
Juegos_iniciados_t
Juegos_iniciados_t_1
Juegos_t
Juegos_t_1
Juegos

In [367]:
empiric_panel_pitcher[['X_Wins_2_t_1']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1789 entries, 0 to 1788
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   X_Wins_2_t_1  1789 non-null   float64
dtypes: float64(1)
memory usage: 14.1 KB


In [368]:
# Rplace infinity values
empiric_panel_pitcher['X_Wins_2_t'] = empiric_panel_pitcher['X_Wins_2_t'].replace(np.inf, np.nan)
empiric_panel_pitcher['X_Wins_t'] = empiric_panel_pitcher['X_Wins_t'].replace(np.inf, np.nan)
empiric_panel_pitcher['X_Wins_2_t_1'] = empiric_panel_pitcher['X_Wins_2_t_1'].replace(np.inf, np.nan)
empiric_panel_pitcher['X_Wins_t_1'] = empiric_panel_pitcher['X_Wins_t_1'].replace(np.inf, np.nan)

In [369]:
empiric_panel_pitcher.to_csv(general_dynamic_path + 'panel_fielders_cum_did_all' + '.csv',
                             index = False)

In [370]:
# Pitcher
for y_name in Y_list:
    for stat in range(0,len(pitcher_regular_stats)):
        # Stat
        print(colored(pitcher_regular_stats[stat], "cyan"))

        # X name
        name = 'X_' + pitcher_regular_stats[stat]

        # OLS variables
        Y = empiric_panel_pitcher[y_name].tolist()
        X = empiric_panel_pitcher[name].tolist()
        X = sm.add_constant(X)

        # Modelo
        model = sm.OLS(Y, X,
                       missing = 'drop').fit()
        print(model.summary())
        print("\n")

Bateos_2_t
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.4533
Date:                Sat, 08 Apr 2023   Prob (F-statistic):              0.501
Time:                        14:37:17   Log-Likelihood:                 878.76
No. Observations:                1789   AIC:                            -1754.
Df Residuals:                    1787   BIC:                            -1743.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0136      0.004      3.8

In [371]:
# Dimentions
print(empiric_panel_hitter.shape)

(1852, 200)


In [372]:
# Dimentions
print(empiric_panel_pitcher.shape)

(1789, 214)
